# TOI-5807.01 (TIC 188646744): End-to-end Validation Walkthrough

This notebook is a fully traceable, end-to-end workflow for evaluating **TOI-5807.01 / TIC 188646744** as a transiting planet candidate using TESS light curves, pixel-level diagnostics, extended robustness checks, and AO-assisted statistical validation.

Design goals:
- **Auditability:** each check is run in its own cell and prints the underlying numerical metrics.
- **Reproducibility:** key artifacts (FPP logs, final validation packet) are written to `persistent_cache/`.
- **Scientifically useful outputs:** the notebook emphasizes reported metrics rather than subjective thresholds.

High-level outline:
1. Load the bundled dataset and define the candidate ephemeris/depth.
2. Run **V01–V15** check-by-check (including pixel checks **V08–V10**).
3. Run extended diagnostics (**V16–V19**) and a per-sector detrending robustness rerun.
4. Compute AO-assisted TRICERATOPS FPP/NFPP and write a consolidated validation packet.


## 1) Target overview

Target:
- **Candidate:** TOI-5807.01
- **Identifier:** TIC 188646744
- **Sky position (used for catalog checks):** RA = 304.12005°, Dec = 11.08344°

Data products used (bundled with this repo):
- Per-sector **PDCSAP** light curves for sectors **55/75/82/83** in `docs/tutorials/data/tic188646744/`.
- A TPF stamp for sector **83** (other sectors may be downloaded if `NETWORK = True`).
- A **PHARO AO contrast curve** (`PHARO_Kcont_plot.tbl`) used to constrain blends in the FPP step.

Key modeling inputs (kept stable for reproducibility):
- Ephemeris: **P = 14.2423724 d**, **t0 = 3540.26317 BTJD**, **duration = 4.046 h**
- Stellar parameters used by duration-related checks: **R⋆ = 1.65 R☉**, **M⋆ = 1.47 M☉**, **Teff = 6816 K**, **log g = 4.17**

What this notebook produces:
- A transparent record of computed metrics/flags for each check.
- A saved TRICERATOPS stdout/stderr log (`persistent_cache/fpp_triceratops_stdout.log`).
- A consolidated JSON packet (`persistent_cache/toi5807_validation_packet.json`).


## 2) Imports + configuration


In [ ]:
from __future__ import annotations

import importlib.metadata
import json

import numpy as np

import bittr_tess_vetter.api as btv

# Controls (edit these)
NETWORK = True  # recommended: allows catalog checks + TPF downloads if missing

TIC_ID = 188646744
TOI_LABEL = 'TOI-5807.01'

# Stable ephemeris defaults (avoid depending on ExoFOP changing over time)
PERIOD_DAYS = 14.2423724
T0_BTJD = 3540.26317
DURATION_HOURS = 4.046

# Stable stellar parameters (used by the duration-consistency check)
STELLAR = btv.StellarParams(radius=1.65, mass=1.47, teff=6816.0, logg=4.17)

# Coordinates used for catalog checks (V06)
RA_DEG = 304.12005
DEC_DEG = 11.08344

# Reduce noisy warnings in notebook outputs
import warnings
warnings.filterwarnings(
    'ignore',
    message=r'Warning: the tpfmodel submodule is not available without oktopus installed',
)


print(
    json.dumps(
        {
            'network': NETWORK,
            'tic_id': TIC_ID,
            'period_days': PERIOD_DAYS,
            't0_btjd': T0_BTJD,
            'duration_hours': DURATION_HOURS,
            'stellar_radius_rsun': STELLAR.radius,
            'stellar_mass_msun': STELLAR.mass,
            'versions': {
                'bittr-tess-vetter': importlib.metadata.version('bittr-tess-vetter'),
                'numpy': importlib.metadata.version('numpy'),
            },
        },
        indent=2,
        sort_keys=True,
    )
)


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "duration_hours": 4.046,
  "network": true,
  "period_days": 14.2423724,
  "stellar_mass_msun": 1.47,
  "stellar_radius_rsun": 1.65,
  "t0_btjd": 3540.26317,
  "tic_id": 188646744,
  "versions": {
    "bittr-tess-vetter": "0.1.0",
    "numpy": "2.3.5"
  }
}
```

</details>


## 3) Load the dataset + define the candidate

We load the per-sector PDCSAP light curves from `docs/tutorials/data/tic188646744`.

We then:
- stitch the sectors into a single time series (for LC-only checks)
- compute a deterministic depth estimate from the stitched light curve (in/out-of-transit median)
- define a `Candidate` object that is reused throughout the notebook


In [ ]:
ds = btv.load_tutorial_target('tic188646744')
print(json.dumps(ds.summary(), indent=2, sort_keys=True))

stitched = btv.stitch_lightcurves(
    [
        {
            'time': np.asarray(lc.time, dtype=np.float64),
            'flux': np.asarray(lc.flux, dtype=np.float64),
            'flux_err': np.asarray(lc.flux_err, dtype=np.float64),
            'sector': int(sector),
            'quality': np.zeros(len(lc.time), dtype=np.int32),
        }
        for sector, lc in sorted(ds.lc_by_sector.items())
    ]
)

in_tr = btv.get_in_transit_mask(stitched.time, PERIOD_DAYS, T0_BTJD, DURATION_HOURS)
out_tr = btv.get_out_of_transit_mask(
    stitched.time,
    PERIOD_DAYS,
    T0_BTJD,
    DURATION_HOURS,
    buffer_factor=3.0,
)
depth_hat, depth_err = btv.measure_transit_depth(stitched.flux, in_tr, out_tr)

DEPTH_PPM = float(depth_hat * 1e6)
DEPTH_ERR_PPM = float(depth_err * 1e6)

ephem = btv.Ephemeris(period_days=PERIOD_DAYS, t0_btjd=T0_BTJD, duration_hours=DURATION_HOURS)
candidate = btv.Candidate(ephemeris=ephem, depth_ppm=DEPTH_PPM)

print(
    json.dumps(
        {
            'tic_id': TIC_ID,
            'period_days': ephem.period_days,
            't0_btjd': ephem.t0_btjd,
            'duration_hours': ephem.duration_hours,
            'depth_ppm_hat': DEPTH_PPM,
            'depth_ppm_err': DEPTH_ERR_PPM,
            'sectors': sorted(ds.lc_by_sector.keys()),
        },
        indent=2,
        sort_keys=True,
    )
)


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "artifacts": [
    "files"
  ],
  "root": "/Users/collier/projects/apps/bittr-tess-vetter/docs/tutorials/data/tic188646744",
  "schema_version": 1,
  "sectors_lc": [
    55,
    75,
    82,
    83
  ],
  "sectors_tpf": [
    83
  ]
}
{
  "depth_ppm_err": 8.445013682052277,
  "depth_ppm_hat": 255.55864733783906,
  "duration_hours": 4.046,
  "period_days": 14.2423724,
  "sectors": [
    55,
    75,
    82,
    83
  ],
  "t0_btjd": 3540.26317,
  "tic_id": 188646744
}
```

</details>


## 4) Run checks one-by-one

We build a reusable `VettingSession` and then run each check individually.

For each check:
- We show the full metrics/flags.
- We provide a short **Analysis** block that interprets the metrics for this target and points to the next step.


In [ ]:
session = btv.VettingSession.from_api(
    lc=btv.LightCurve(
        time=stitched.time,
        flux=stitched.flux,
        flux_err=stitched.flux_err,
        quality=stitched.quality,
    ),
    candidate=candidate,
    stellar=STELLAR,
    network=False,
    preset='default',
    tic_id=TIC_ID,
)

print(json.dumps({'session': 'ready', 'checks': ['V01','V02','V03','V04','V05']}, indent=2))


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "session": "ready",
  "checks": [
    "V01",
    "V02",
    "V03",
    "V04",
    "V05"
  ]
}
```

</details>


### V01 — Odd/Even depth comparison


In [ ]:
r = session.run('V01')
print(json.dumps(r.model_dump(), indent=2, sort_keys=True))
print(btv.format_check_result(r, max_metrics=25))


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "confidence": 0.7,
  "flags": [],
  "id": "V01",
  "metrics": {
    "delta_ppm": -68.6,
    "delta_sigma": 1.73,
    "depth_diff_sigma": 1.73,
    "depth_err_even_ppm": 30.6,
    "depth_err_odd_ppm": 25.2,
    "depth_even_ppm": 297.6,
    "depth_odd_ppm": 229.0,
    "even_depth": 0.000298,
    "method": "per_epoch_median",
    "n_even_points": 487,
    "n_even_transits": 4,
    "n_odd_points": 485,
    "n_odd_transits": 4,
    "odd_depth": 0.000229,
    "rel_diff": 0.23
  },
  "name": "Odd-Even Depth",
  "notes": [],
  "provenance": {},
  "raw": {
    "_metrics_only": true,
    "delta_ppm": -68.6,
    "delta_sigma": 1.73,
    "depth_diff_sigma": 1.73,
    "depth_err_even_ppm": 30.6,
    "depth_err_odd_ppm": 25.2,
    "depth_even_ppm": 297.6,
    "depth_odd_ppm": 229.0,
    "epoch_depths_even_ppm": [
      280.6,
      356.4,
      191.8,
      314.6
    ],
    "epoch_depths_odd_ppm": [
      156.2,
      226.8,
      231.2,
      266.2
    ],
    "even_depth": 0.000298,
    "method": "per_epoch_median",
    "n_even_points": 487,
    "n_even_transits": 4,
    "n_odd_points": 485,
    "n_odd_transits": 4,
    "odd_depth": 0.000229,
    "rel_diff": 0.23
  },
  "status": "ok"
}
Check Result
============================================================
ID          V01
Name        Odd-Even Depth
Status      ok
Confidence  0.700

Metrics
------------------------------------------------------------
delta_ppm           -68.6
delta_sigma         1.73
depth_diff_sigma    1.73
depth_err_even_ppm  30.6
depth_err_odd_ppm   25.2
depth_even_ppm      297.6
depth_odd_ppm       229.0
even_depth          0.000298
method              per_epoch_median
n_even_points       487
n_even_transits     4
n_odd_points        485
n_odd_transits      4
odd_depth           0.000229
rel_diff            0.23
```

</details>


<details>
<summary><b>Analysis</b></summary>

- **Result:** odd vs even depths differ by **1.73σ** (odd≈229.0 ppm, even≈297.6 ppm; Δ≈-68.6 ppm).
- **Interpretation:** This is not a compelling odd/even mismatch; it does **not** suggest an eclipsing binary at 2× the reported period.
- **Next check:** run **V02 (secondary eclipse)** because a significant eclipse near phase 0.5 is a classic EB indicator that is independent of odd/even behavior.

</details>


### V02 — Secondary eclipse search (phase ~0.5)


In [ ]:
r = session.run('V02')
print(json.dumps(r.model_dump(), indent=2, sort_keys=True))
print(btv.format_check_result(r, max_metrics=25))


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "confidence": 0.85,
  "flags": [],
  "id": "V02",
  "metrics": {
    "baseline_flux": 1.000006,
    "n_baseline_points": 21177,
    "n_secondary_events_effective": 8,
    "n_secondary_points": 22130,
    "red_noise_inflation": 4.84,
    "secondary_depth": 9e-06,
    "secondary_depth_err_ppm": 8.43,
    "secondary_depth_ppm": 9.4,
    "secondary_depth_sigma": 1.12,
    "secondary_phase_coverage": 1.0
  },
  "name": "Secondary Eclipse",
  "notes": [],
  "provenance": {},
  "raw": {
    "_metrics_only": true,
    "baseline_flux": 1.000006,
    "n_baseline_points": 21177,
    "n_secondary_events_effective": 8,
    "n_secondary_points": 22130,
    "red_noise_inflation": 4.84,
    "search_window": [
      0.35,
      0.65
    ],
    "secondary_depth": 9e-06,
    "secondary_depth_err_ppm": 8.43,
    "secondary_depth_ppm": 9.4,
    "secondary_depth_sigma": 1.12,
    "secondary_phase_coverage": 1.0
  },
  "status": "ok"
}
Check Result
============================================================
ID          V02
Name        Secondary Eclipse
Status      ok
Confidence  0.850

Metrics
------------------------------------------------------------
baseline_flux                 1.000006
n_baseline_points             21177
n_secondary_events_effective  8
n_secondary_points            22130
red_noise_inflation           4.84
secondary_depth               9e-06
secondary_depth_err_ppm       8.43
secondary_depth_ppm           9.4
secondary_depth_sigma         1.12
secondary_phase_coverage      1.0
```

</details>


<details>
<summary><b>Analysis</b></summary>

- **Result:** secondary depth ≈ **9.4 ± 8.43 ppm** (**1.12σ**) with phase coverage **1.0** (good coverage).
- **Interpretation:** No significant secondary eclipse is detected; this supports a planet-like interpretation (or at least argues against a luminous EB secondary).
- **Next check:** run **V03 (duration consistency)** as a physics sanity check using the assumed stellar parameters.

</details>


### V03 — Duration consistency (stellar-density sanity check)


In [ ]:
r = session.run('V03')
print(json.dumps(r.model_dump(), indent=2, sort_keys=True))
print(btv.format_check_result(r, max_metrics=25))


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "confidence": 0.85,
  "flags": [],
  "id": "V03",
  "metrics": {
    "density_corrected": true,
    "duration_hours": 4.046,
    "duration_ratio": 0.632,
    "expected_duration_hours": 6.3971,
    "expected_duration_solar": 4.4083,
    "period_days": 14.2424,
    "stellar_density_solar": 0.3272,
    "stellar_mass": 1.47,
    "stellar_radius": 1.65
  },
  "name": "Duration Consistency",
  "notes": [],
  "provenance": {},
  "raw": {
    "_metrics_only": true,
    "density_corrected": true,
    "duration_hours": 4.046,
    "duration_ratio": 0.632,
    "expected_duration_hours": 6.3971,
    "expected_duration_solar": 4.4083,
    "period_days": 14.2424,
    "stellar_density_solar": 0.3272,
    "stellar_mass": 1.47,
    "stellar_radius": 1.65
  },
  "status": "ok"
}
Check Result
============================================================
ID          V03
Name        Duration Consistency
Status      ok
Confidence  0.850

Metrics
------------------------------------------------------------
density_corrected        True
duration_hours           4.046
duration_ratio           0.632
expected_duration_hours  6.3971
expected_duration_solar  4.4083
period_days              14.2424
stellar_density_solar    0.3272
stellar_mass             1.47
stellar_radius           1.65
```

</details>


<details>
<summary><b>Analysis</b></summary>

- **Result:** observed duration **4.046 h** vs expected **6.3971 h** (ratio **0.632**) using the provided stellar parameters.
- **Interpretation:** A shorter-than-expected duration can be produced by geometry (higher impact parameter) and/or eccentricity, or by stellar-parameter mismatch. This is a **guardrail**, not a planet/FP verdict.
- **Next check:** run **V04 (depth stability)** to see whether the depth is consistent across individual epochs (and to motivate per-sector robustness if it isn’t).

</details>


### V04 — Depth stability (epoch-to-epoch consistency)


In [ ]:
r = session.run('V04')
print(json.dumps(r.model_dump(), indent=2, sort_keys=True))
print(btv.format_check_result(r, max_metrics=25))


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "confidence": 0.7,
  "flags": [],
  "id": "V04",
  "metrics": {
    "chi2_reduced": 1.34,
    "d_mean_w_ppm": 248.12,
    "d_med_ppm": 248.73,
    "depth_mad_ppm": 44.39,
    "depth_scatter_ppm": 60.9,
    "dmm": 0.998,
    "dmm_abs": 0.002,
    "dom_frac": 0.217,
    "dom_ratio": 1.318,
    "dominating_epoch_index": 3,
    "dominating_epoch_time_btjd": 3582.990287,
    "expected_scatter_ppm": 19.93,
    "mean_depth": 0.000253,
    "mean_depth_ppm": 253.0,
    "method": "per_epoch_local_baseline",
    "n_transits_measured": 8,
    "rms_scatter": 0.2409,
    "s_max": 6.156,
    "s_median": 4.681,
    "std_depth": 6.1e-05
  },
  "name": "Depth Stability",
  "notes": [],
  "provenance": {},
  "raw": {
    "_metrics_only": true,
    "chi2_reduced": 1.34,
    "d_mean_w_ppm": 248.12,
    "d_med_ppm": 248.73,
    "depth_mad_ppm": 44.39,
    "depth_scatter_ppm": 60.9,
    "depths_ppm": [
      280.6,
      156.2,
      356.4,
      226.8,
      191.8,
      231.2,
      314.6,
      266.2
    ],
    "dmm": 0.998,
    "dmm_abs": 0.002,
    "dom_frac": 0.217,
    "dom_ratio": 1.318,
    "dominating_epoch_index": 3,
    "dominating_epoch_time_btjd": 3582.990287,
    "expected_scatter_ppm": 19.93,
    "individual_depths": [
      0.000281,
      0.000156,
      0.000356,
      0.000227,
      0.000192,
      0.000231,
      0.000315,
      0.000266
    ],
    "mean_depth": 0.000253,
    "mean_depth_ppm": 253.0,
    "method": "per_epoch_local_baseline",
    "n_transits_measured": 8,
    "outlier_epochs": [],
    "rms_scatter": 0.2409,
    "s_max": 6.156,
    "s_median": 4.681,
    "std_depth": 6.1e-05
  },
  "status": "ok"
}
Check Result
============================================================
ID          V04
Name        Depth Stability
Status      ok
Confidence  0.700

Metrics
------------------------------------------------------------
chi2_reduced                1.34
d_mean_w_ppm                248.12
d_med_ppm                   248.73
depth_mad_ppm               44.39
depth_scatter_ppm           60.9
dmm                         0.998
dmm_abs                     0.002
dom_frac                    0.217
dom_ratio                   1.318
dominating_epoch_index      3
dominating_epoch_time_btjd  3582.990287
expected_scatter_ppm        19.93
mean_depth                  0.000253
mean_depth_ppm              253.0
method                      per_epoch_local_baseline
n_transits_measured         8
rms_scatter                 0.2409
s_max                       6.156
s_median                    4.681
std_depth                   6.1e-05
```

</details>


<details>
<summary><b>Analysis</b></summary>

- **Result:** per-epoch depth scatter ≈ **60.9 ppm** vs expected ≈ **19.93 ppm** (reduced χ² **1.34**) across **8** measured transits.
- **Interpretation:** Depth varies more than simple white-noise expectations. This can reflect systematics, residual variability, or real depth scatter; it’s a prompt to examine per-sector behavior and robustness, not an immediate invalidation.
- **Next checks (sequential):** run **V05** next. If V05 is transit-like, proceed **V06 → V07 → V08 → V09 → V10**, then **V11 → V11b → V12 → V13 → V15**.

</details>


### V05 — Transit shape (U vs V)


In [ ]:
r = session.run('V05')
print(json.dumps(r.model_dump(), indent=2, sort_keys=True))
print(btv.format_check_result(r, max_metrics=25))


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "confidence": 0.935,
  "flags": [],
  "id": "V05",
  "metrics": {
    "depth_bottom": 0.000299,
    "depth_edge": 0.000235,
    "depth_ppm": 256.5,
    "method": "trapezoid_grid_search",
    "n_baseline": 35680,
    "n_bottom_points": 243,
    "n_edge_points": 488,
    "n_in_transit": 1165,
    "shape_metric_uncertainty": 0.0,
    "shape_ratio": 1.272,
    "status": "ok",
    "t_flat_hours": 4.046,
    "t_total_hours": 4.046,
    "tflat_ttotal_ratio": 1.0,
    "tflat_ttotal_ratio_err": 0.0,
    "transit_coverage": 1.0
  },
  "name": "V-Shape",
  "notes": [],
  "provenance": {},
  "raw": {
    "_metrics_only": true,
    "depth_bottom": 0.000299,
    "depth_edge": 0.000235,
    "depth_ppm": 256.5,
    "method": "trapezoid_grid_search",
    "n_baseline": 35680,
    "n_bottom_points": 243,
    "n_edge_points": 488,
    "n_in_transit": 1165,
    "shape_metric_uncertainty": 0.0,
    "shape_ratio": 1.272,
    "status": "ok",
    "t_flat_hours": 4.046,
    "t_total_hours": 4.046,
    "tflat_ttotal_ratio": 1.0,
    "tflat_ttotal_ratio_err": 0.0,
    "transit_coverage": 1.0
  },
  "status": "ok"
}
Check Result
============================================================
ID          V05
Name        V-Shape
Status      ok
Confidence  0.935

Metrics
------------------------------------------------------------
depth_bottom              0.000299
depth_edge                0.000235
depth_ppm                 256.5
method                    trapezoid_grid_search
n_baseline                35680
n_bottom_points           243
n_edge_points             488
n_in_transit              1165
shape_metric_uncertainty  0.0
shape_ratio               1.272
status                    ok
t_flat_hours              4.046
t_total_hours             4.046
tflat_ttotal_ratio        1.0
tflat_ttotal_ratio_err    0.0
transit_coverage          1.0
```

</details>


<details>
<summary><b>Analysis</b></summary>

- **Result:** trapezoid fit gives **tflat/ttotal = 1.0** (t_flat = t_total = 4.046 h) with **shape_ratio = 1.272**, depth ≈ **256.5 ppm** (depth_bottom≈2.99×10⁻⁴, depth_edge≈2.35×10⁻⁴).
- **Interpretation:** This is **U-shaped / flat-bottomed** rather than V-shaped, which is more consistent with a planet transit than a grazing eclipsing binary.
- **Caveat:** a clean U-shape does **not** prove on-target origin; blends can still produce planet-like shapes after dilution.
- **Next checks (sequential):** **V06 → V07 → V08 → V09 → V10**, then exovetter (V11/V12/V11b) and AO-assisted FPP.

</details>


## 5) Catalog context checks (V06–V07)

These checks use external catalogs (network required):
- **V06**: search nearby eclipsing binaries that could contaminate the TESS aperture.
- **V07**: ExoFOP TOI lookup for community/archival context.


In [ ]:
if not NETWORK:
    raise ValueError('Set NETWORK=True to run catalog checks')

session_catalog = btv.VettingSession.from_api(
    lc=btv.LightCurve(
        time=stitched.time,
        flux=stitched.flux,
        flux_err=stitched.flux_err,
        quality=stitched.quality,
    ),
    candidate=candidate,
    stellar=STELLAR,
    network=True,
    ra_deg=RA_DEG,
    dec_deg=DEC_DEG,
    tic_id=TIC_ID,
    preset='default',
)

print(json.dumps({'catalog_session': 'ready', 'ra_deg': RA_DEG, 'dec_deg': DEC_DEG, 'tic_id': TIC_ID}, indent=2))


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "catalog_session": "ready",
  "ra_deg": 304.12005,
  "dec_deg": 11.08344,
  "tic_id": 188646744
}
```

</details>


### V06 — Nearby eclipsing binary search (catalog)

In [ ]:
r = session_catalog.run('V06')
print(json.dumps(r.model_dump(), indent=2, sort_keys=True))
print(btv.format_check_result(r, max_metrics=25))


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "confidence": 0.6,
  "flags": [],
  "id": "V06",
  "metrics": {
    "dec": 11.08344,
    "n_ebs_found": 0,
    "ra": 304.12005,
    "search_radius_arcsec": 42.0,
    "status": "ok"
  },
  "name": "Nearby EB Search",
  "notes": [],
  "provenance": {},
  "raw": {
    "_metrics_only": true,
    "dec": 11.08344,
    "n_ebs_found": 0,
    "ra": 304.12005,
    "search_radius_arcsec": 42.0,
    "status": "ok"
  },
  "status": "ok"
}
Check Result
============================================================
ID          V06
Name        Nearby EB Search
Status      ok
Confidence  0.600

Metrics
------------------------------------------------------------
dec                   11.08344
n_ebs_found           0
ra                    304.12005
search_radius_arcsec  42.0
status                ok
```

</details>


<details>
<summary><b>Analysis</b></summary>

- **Result:** `n_ebs_found = 0` within the configured search radius (42″).
- **Interpretation:** No *known* nearby eclipsing binaries are flagged as contaminants by this catalog query. This does not rule out unknown/unlisted EBs, but it reduces one common false-positive mode.
- **Next check:** run **V07** for ExoFOP TOI context (known status, disposition history, and follow-up links).

</details>


### V07 — ExoFOP TOI lookup (context)

In [ ]:
r = session_catalog.run('V07')
print(json.dumps(r.model_dump(), indent=2, sort_keys=True))
print(btv.format_check_result(r, max_metrics=25))


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "confidence": 0.8,
  "flags": [
    "EXOFOP_MATCH"
  ],
  "id": "V07",
  "metrics": {
    "cache_age_seconds": 10034.348549365997,
    "cache_stale": false,
    "cache_ttl_seconds": 86400,
    "fetch_error": null,
    "found": true,
    "source": "exofop_toi_table",
    "status": "ok",
    "tic_id": 188646744,
    "toi": null,
    "used_stale_cache": false
  },
  "name": "ExoFOP TOI Lookup",
  "notes": [
    "Candidate found in ExoFOP TOI table"
  ],
  "provenance": {},
  "raw": {
    "_metrics_only": true,
    "cache_age_seconds": 10034.348549365997,
    "cache_stale": false,
    "cache_ttl_seconds": 86400,
    "fetch_error": null,
    "found": true,
    "row": {
      "comments": "slight depth aperture correlation",
      "date_modified": "2025-07-31 12:59:22",
      "date_toi_alerted_utc": "2022-10-12",
      "date_toi_updated_utc": "2024-11-13",
      "dec": "43:21:38.48",
      "depth_mmag": "0.244318",
      "depth_mmag_err": "0.014569",
      "depth_ppm": "225",
      "depth_ppm_err": "13.4182",
      "detection": "QLP",
      "duration_hours": "4.046",
      "duration_hours_err": "1.263",
      "epoch_bjd": "2460540.26317",
      "epoch_bjd_err": "0.0055979",
      "esm": "3.1",
      "imaging_observations": "2",
      "master": "2",
      "period_days": "14.2423724",
      "period_days_err": "0.0000855",
      "pipeline_signal_id": "1",
      "planet_equil_temp_k": "1027",
      "planet_insolation_earth_flux": "184.841",
      "planet_name": "",
      "planet_radius_r_earth": "2.12067",
      "planet_radius_r_earth_err": ".131587",
      "planet_snr": "19",
      "pm_dec_err_mas_yr": "0.049",
      "pm_dec_mas_yr": "-77.175",
      "pm_ra_err_mas_yr": "0.045",
      "pm_ra_mas_yr": "14.776",
      "predicted_mass_m_earth": "5.14",
      "predicted_rv_semi_amplitude_m_s": "1",
      "previous_ctoi": "",
      "ra": "20:34:16.13",
      "sectors": "15,41,55,56,75,82",
      "sg1a": "4",
      "sg1b": "2",
      "sg2": "2",
      "sg3": "4",
      "sg4": "4",
      "sg5": "4",
      "source": "qlp-s82-ffi",
      "spectroscopy_observations": "2",
      "stellar_distance_pc": "72.9611",
      "stellar_distance_pc_err": ".14735",
      "stellar_eff_temp_k": "6815.9",
      "stellar_eff_temp_k_err": "110.6",
      "stellar_log_g_cm_s^2": "4.17",
      "stellar_log_g_cm_s^2_err": ".08",
      "stellar_mass_m_sun": "1.47",
      "stellar_mass_m_sun_err": ".239126",
      "stellar_metallicity": "",
      "stellar_metallicity_err": "",
      "stellar_radius_r_sun": "1.65",
      "stellar_radius_r_sun_err": ".06",
      "tess_disposition": "PC",
      "tess_mag": "6.8806",
      "tess_mag_err": "0.006",
      "tfopwg_disposition": "PC",
      "tic_id": "188646744",
      "time_series_observations": "0",
      "toi": "5807.01",
      "tsm": "50.4"
    },
    "source": "exofop_toi_table",
    "status": "ok",
    "tic_id": 188646744,
    "toi": null,
    "used_stale_cache": false
  },
  "status": "ok"
}
Check Result
============================================================
ID          V07
Name        ExoFOP TOI Lookup
Status      ok
Confidence  0.800

Metrics
------------------------------------------------------------
cache_age_seconds  10034.348549365997
cache_stale        False
cache_ttl_seconds  86400
fetch_error
found              True
source             exofop_toi_table
status             ok
tic_id             188646744
toi
used_stale_cache   False

Flags
------------------------------------------------------------
- EXOFOP_MATCH

Notes
------------------------------------------------------------
- Candidate found in ExoFOP TOI table
```

</details>


<details>
<summary><b>Analysis</b></summary>

- **Result:** ExoFOP lookup `found = true` (flag `EXOFOP_MATCH`).
- **Interpretation:** This is a known TOI target; ExoFOP provides the community context and follow-up artifacts.
- **Next checks:** proceed to **pixel localization** sequentially (**V08 → V09 → V10**) using per-sector TPFs matched to the per-sector light curves.

</details>


## 6) Pixel localization checks (V08–V10)

These require per-sector TPFs that match the per-sector light curves.

We load cached TPF stamps when available; otherwise we download via `lightkurve` (network required).


In [ ]:
from pathlib import Path

import numpy as _np

# Start with any bundled TPF stamps (the tutorial dataset includes sector 83).
tpf_by_sector: dict[int, btv.TPFStamp] = {int(k): v for k, v in ds.tpf_by_sector.items()}

cache_dir = Path('persistent_cache/toi5807_tpfs')
cache_dir.mkdir(parents=True, exist_ok=True)

# Try disk cache first (so reruns do not require downloads)
for sector in sorted(ds.lc_by_sector.keys()):
    sector = int(sector)
    if sector in tpf_by_sector:
        continue
    npz_path = cache_dir / f'sector{sector}_tpf.npz'
    if not npz_path.exists():
        continue
    d = _np.load(npz_path, allow_pickle=True)
    try:
        from astropy.wcs import WCS

        wcs = WCS(d['wcs_header'].item()) if 'wcs_header' in d else None
    except Exception:
        wcs = d['wcs_header'].item() if 'wcs_header' in d else None

    tpf_by_sector[sector] = btv.TPFStamp(
        time=_np.asarray(d['time'], dtype=_np.float64),
        flux=_np.asarray(d['flux'], dtype=_np.float64),
        flux_err=_np.asarray(d['flux_err'], dtype=_np.float64) if 'flux_err' in d else None,
        wcs=wcs,
        aperture_mask=_np.asarray(d['aperture_mask'], dtype=bool) if 'aperture_mask' in d else None,
        quality=_np.asarray(d['quality'], dtype=_np.int32) if 'quality' in d else None,
    )

# Download any missing sectors if allowed
missing = [int(s) for s in sorted(ds.lc_by_sector.keys()) if int(s) not in tpf_by_sector]
if missing:
    if not NETWORK:
        raise ValueError(f'Missing TPFs for sectors {missing}; set NETWORK=True to download')
    import lightkurve as lk

    for sector in missing:
        search = lk.search_targetpixelfile(f'TIC {TIC_ID}', sector=int(sector), exptime=120)
        if len(search) == 0:
            print(f'No TPF for sector {sector}')
            continue
        tpf = search.download()
        if tpf is None:
            print(f'Download failed for sector {sector}')
            continue

        stamp = btv.TPFStamp(
            time=_np.asarray(tpf.time.value, dtype=_np.float64),
            flux=_np.asarray(tpf.flux.value, dtype=_np.float64),
            flux_err=_np.asarray(tpf.flux_err.value, dtype=_np.float64),
            wcs=tpf.wcs,
            aperture_mask=_np.asarray(tpf.pipeline_mask, dtype=bool),
            quality=_np.asarray(tpf.quality, dtype=_np.int32),
        )
        tpf_by_sector[int(sector)] = stamp

        # Cache to disk
        try:
            wcs_header = dict(tpf.wcs.to_header()) if getattr(tpf, 'wcs', None) is not None else None
            _np.savez(
                cache_dir / f'sector{sector}_tpf.npz',
                time=stamp.time,
                flux=stamp.flux,
                flux_err=stamp.flux_err,
                wcs_header=wcs_header,
                aperture_mask=stamp.aperture_mask,
                quality=stamp.quality,
            )
        except Exception:
            pass

print(json.dumps({'tpf_sectors_loaded': sorted(tpf_by_sector.keys()), 'tpf_cache_dir': str(cache_dir)}, indent=2, sort_keys=True))


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "tpf_cache_dir": "persistent_cache/toi5807_tpfs",
  "tpf_sectors_loaded": [
    55,
    75,
    82,
    83
  ]
}
```

</details>


### V08 — Centroid shift (per sector)

In [ ]:
out = []
for sector in sorted(ds.lc_by_sector.keys()):
    sector = int(sector)
    sec_lc = ds.lc_by_sector[sector]
    sec_tpf = tpf_by_sector.get(sector)

    session_sec = btv.VettingSession.from_api(
        lc=btv.LightCurve(time=sec_lc.time, flux=sec_lc.flux, flux_err=sec_lc.flux_err),
        candidate=candidate,
        stellar=STELLAR,
        tpf=sec_tpf,
        network=False,
        preset='default',
        tic_id=TIC_ID,
    )
    r = session_sec.run('V08')
    out.append({'sector': sector, 'status': r.status, 'centroid_shift_pixels': r.metrics.get('centroid_shift_pixels'), 'significance_sigma': r.metrics.get('significance_sigma'), 'flags': r.flags})

print(json.dumps(out, indent=2, sort_keys=True))


<details>
<summary><b>Expected Output</b></summary>

```text
[
  {
    "centroid_shift_pixels": 0.0037815144090368935,
    "flags": [],
    "sector": 55,
    "significance_sigma": 0.4551653707150233,
    "status": "ok"
  },
  {
    "centroid_shift_pixels": 0.014452302897329383,
    "flags": [],
    "sector": 75,
    "significance_sigma": 1.8703471299915302,
    "status": "ok"
  },
  {
    "centroid_shift_pixels": 0.004299943855965851,
    "flags": [],
    "sector": 82,
    "significance_sigma": 1.5693309176012056,
    "status": "ok"
  },
  {
    "centroid_shift_pixels": 0.017002980203089595,
    "flags": [],
    "sector": 83,
    "significance_sigma": 3.2554430353630757,
    "status": "ok"
  }
]
```

</details>


<details>
<summary><b>Analysis</b></summary>

- **Result (per sector):** centroid shifts are small (≈0.0038–0.0170 pixels) with significance ranging from ≈0.46σ up to ≈3.23σ.
- **Interpretation:** This does not show a strong, consistent off-target centroid shift. The sector-to-sector variation motivates checking the stronger spatial diagnostic next.
- **Next check:** run **V09** (difference image localization) to see where the in-transit signal is concentrated in the pixel stamp.

</details>


### V09 — Difference image localization (per sector)

In [ ]:
out = []
for sector in sorted(ds.lc_by_sector.keys()):
    sector = int(sector)
    sec_lc = ds.lc_by_sector[sector]
    sec_tpf = tpf_by_sector.get(sector)

    session_sec = btv.VettingSession.from_api(
        lc=btv.LightCurve(time=sec_lc.time, flux=sec_lc.flux, flux_err=sec_lc.flux_err),
        candidate=candidate,
        stellar=STELLAR,
        tpf=sec_tpf,
        network=False,
        preset='default',
        tic_id=TIC_ID,
    )
    r = session_sec.run('V09')
    out.append(
        {
            'sector': sector,
            'status': r.status,
            'distance_to_target_pixels': r.metrics.get('distance_to_target_pixels'),
            'concentration_ratio': r.metrics.get('concentration_ratio'),
            'target_depth_ppm': r.metrics.get('target_depth_ppm'),
            'max_depth_ppm': r.metrics.get('max_depth_ppm'),
            'target_pixel_row': r.metrics.get('target_pixel_row'),
            'target_pixel_col': r.metrics.get('target_pixel_col'),
            'max_depth_pixel_row': r.metrics.get('max_depth_pixel_row'),
            'max_depth_pixel_col': r.metrics.get('max_depth_pixel_col'),
            'lc_tpf_time_overlap_days': r.metrics.get('lc_tpf_time_overlap_days'),
            'flags': r.flags,
        }
    )

print(json.dumps(out, indent=2, sort_keys=True))


<details>
<summary><b>Expected Output</b></summary>

```text
[
  {
    "concentration_ratio": 0.0019413074671024704,
    "distance_to_target_pixels": 7.211102550927978,
    "flags": [
      "DIFFIMG_MAX_AT_EDGE",
      "DIFFIMG_UNRELIABLE"
    ],
    "lc_tpf_time_overlap_days": 27.16260021847893,
    "max_depth_pixel_col": 9,
    "max_depth_pixel_row": 0,
    "max_depth_ppm": 3721467.367126439,
    "sector": 55,
    "status": "ok",
    "target_depth_ppm": 7224.512388380727,
    "target_pixel_col": 5,
    "target_pixel_row": 6
  },
  {
    "concentration_ratio": -0.046365310779727525,
    "distance_to_target_pixels": 7.810249675906654,
    "flags": [
      "DIFFIMG_MAX_AT_EDGE",
      "DIFFIMG_TARGET_DEPTH_NONPOSITIVE",
      "DIFFIMG_UNRELIABLE"
    ],
    "lc_tpf_time_overlap_days": 27.697002671326118,
    "max_depth_pixel_col": 10,
    "max_depth_pixel_row": 0,
    "max_depth_ppm": 557510.2548859358,
    "sector": 75,
    "status": "ok",
    "target_depth_ppm": -25849.13623067152,
    "target_pixel_col": 5,
    "target_pixel_row": 6
  },
  {
    "concentration_ratio": -0.014653067156932462,
    "distance_to_target_pixels": 7.0710678118654755,
    "flags": [
      "DIFFIMG_MAX_AT_EDGE",
      "DIFFIMG_TARGET_DEPTH_NONPOSITIVE",
      "DIFFIMG_UNRELIABLE"
    ],
    "lc_tpf_time_overlap_days": 25.85275969825034,
    "max_depth_pixel_col": 10,
    "max_depth_pixel_row": 1,
    "max_depth_ppm": 546367.7780012799,
    "sector": 82,
    "status": "ok",
    "target_depth_ppm": -8005.963743436721,
    "target_pixel_col": 5,
    "target_pixel_row": 6
  },
  {
    "concentration_ratio": -0.029692563072028194,
    "distance_to_target_pixels": 6.0,
    "flags": [
      "DIFFIMG_MAX_AT_EDGE",
      "DIFFIMG_TARGET_DEPTH_NONPOSITIVE",
      "DIFFIMG_UNRELIABLE"
    ],
    "lc_tpf_time_overlap_days": 24.918892079784655,
    "max_depth_pixel_col": 5,
    "max_depth_pixel_row": 0,
    "max_depth_ppm": 224066.93494969374,
    "sector": 83,
    "status": "ok",
    "target_depth_ppm": -6653.12159834982,
    "target_pixel_col": 5,
    "target_pixel_row": 6
  }
]
```

</details>


<details>
<summary><b>Analysis</b></summary>

- **Result (per sector):** `distance_to_target_pixels` is large (≈6–8 px) and `target_depth_ppm` is negative in multiple sectors, while `max_depth_ppm` occurs near the stamp edge.
- **Interpretation:** This pattern suggests the difference-image metric is dominated by a non-target pixel pattern (edge/bleed/systematic), rather than a clean on-target transit localization. For a very bright star, saturation/bleed and background systematics can make naive difference-image localization fragile.
- **Next check:** run **V10** (aperture dependence) to see whether the measured depth is stable vs aperture size, and then (in the next batch) use per-sector robustness/diagnostic plots to understand why V09 localizes off-target.

</details>


### V10 — Aperture dependence (per sector)

In [ ]:
out = []
for sector in sorted(ds.lc_by_sector.keys()):
    sector = int(sector)
    sec_lc = ds.lc_by_sector[sector]
    sec_tpf = tpf_by_sector.get(sector)

    session_sec = btv.VettingSession.from_api(
        lc=btv.LightCurve(time=sec_lc.time, flux=sec_lc.flux, flux_err=sec_lc.flux_err),
        candidate=candidate,
        stellar=STELLAR,
        tpf=sec_tpf,
        network=False,
        preset='default',
        tic_id=TIC_ID,
    )
    r = session_sec.run('V10')
    out.append(
        {
            'sector': sector,
            'status': r.status,
            'stability_metric': r.metrics.get('stability_metric'),
            'depth_variance_ppm2': r.metrics.get('depth_variance_ppm2'),
            'recommended_aperture_pixels': r.metrics.get('recommended_aperture_pixels'),
            'depth_ppm_aperture_1p0': r.metrics.get('depth_ppm_aperture_1p0'),
            'depth_ppm_aperture_3p0': r.metrics.get('depth_ppm_aperture_3p0'),
            'flags': r.flags,
        }
    )

print(json.dumps(out, indent=2, sort_keys=True))


<details>
<summary><b>Expected Output</b></summary>

```text
[
  {
    "depth_ppm_aperture_1p0": 199.07785479117422,
    "depth_ppm_aperture_3p0": 200.5432275224983,
    "depth_variance_ppm2": 61.93261645250078,
    "flags": [],
    "recommended_aperture_pixels": 2.0,
    "sector": 55,
    "stability_metric": 0.9476333172012621,
    "status": "ok"
  },
  {
    "depth_ppm_aperture_1p0": -8.56705518570422,
    "depth_ppm_aperture_3p0": 244.16930738530016,
    "depth_variance_ppm2": 8698.152366206981,
    "flags": [
      "negative_depths_present"
    ],
    "recommended_aperture_pixels": 2.0,
    "sector": 75,
    "stability_metric": 0.4001345694379497,
    "status": "ok"
  },
  {
    "depth_ppm_aperture_1p0": 335.9793246395237,
    "depth_ppm_aperture_3p0": 154.63275974902936,
    "depth_variance_ppm2": 4437.305770408408,
    "flags": [],
    "recommended_aperture_pixels": 1.5,
    "sector": 82,
    "stability_metric": 0.6371928139759866,
    "status": "ok"
  },
  {
    "depth_ppm_aperture_1p0": 240.0796456475951,
    "depth_ppm_aperture_3p0": 249.4213885769203,
    "depth_variance_ppm2": 162.71352257120816,
    "flags": [],
    "recommended_aperture_pixels": 2.0,
    "sector": 83,
    "stability_metric": 0.9280707188941079,
    "status": "ok"
  }
]
```

</details>


<details>
<summary><b>Analysis</b></summary>

- **Result (per sector):** depth vs aperture behavior is mixed. Example: sector 75 shows an unstable/signed-flipping small-aperture depth (`depth_ppm_aperture_1p0` negative) and a low `stability_metric`, while other sectors are more stable.
- **Interpretation:** This is a prompt to treat pixel-level diagnostics with care for this bright target and to rely on multi-sector consistency, robust detrending/normalization, and additional diagnostics before making a strong on-target claim from pixels alone.
- **Next checks (sequential):** run **V11 → V11b → V12 → V13 → V15**, then AO-assisted FPP.

</details>


## 7) Exovetter-based checks (V11, V11b, V12)

These checks use **ExoVetter-style** algorithms to test whether the detected signal is unique and transit-like:
- **V11 (ModShift):** look for competing signals at other phases (secondary/tertiary events).
- **V11b (ModShiftUniqueness):** an independent ModShift-style implementation with additional uniqueness metrics.
- **V12 (SWEET):** check for stellar variability at the transit period (or half/double) that could mimic a transit.


### V11 — ModShift (signal uniqueness)


In [ ]:
r = session.run('V11')
print(json.dumps(r.model_dump(), indent=2, sort_keys=True))
print(btv.format_check_result(r, max_metrics=25))


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "confidence": 1.0,
  "flags": [],
  "id": "V11",
  "metrics": {
    "baseline_days": 787.27,
    "false_alarm_threshold": 2.516974,
    "fred": 54.286904,
    "n_points": 73805,
    "n_transits_expected": 55,
    "positive_primary_ratio": 0.0,
    "positive_signal": 784.0,
    "primary_signal": 0.0,
    "secondary_primary_ratio": 0.0,
    "secondary_signal": 21.0,
    "tertiary_primary_ratio": 0.0,
    "tertiary_signal": 726.0
  },
  "name": "ModShift",
  "notes": [],
  "provenance": {},
  "raw": {
    "_metrics_only": true,
    "false_alarm_threshold": 2.516974,
    "fred": 54.286904,
    "inputs_summary": {
      "baseline_days": 787.27,
      "cadence_median_min": 2.0,
      "flux_err_available": true,
      "n_points": 73805,
      "n_transits_expected": 55
    },
    "positive_primary_ratio": 0.0,
    "positive_signal": 784.0,
    "positive_signal_signed": 784.0,
    "primary_signal": 0.0,
    "primary_signal_signed": 0.0,
    "raw_metrics": {
      "Fred": 54.28690391454527,
      "false_alarm_threshold": 2.5169743319493008,
      "phase_pos": 13.22988147109005,
      "phase_pri": 0.0,
      "phase_sec": 0.3543718251184834,
      "phase_ter": 12.251140239810429,
      "pos": 784,
      "pri": 0,
      "sec": 21,
      "sigma_pos": 2266.986950650904,
      "sigma_pri": -6153.077969439587,
      "sigma_sec": -1510.4768369680066,
      "sigma_ter": -1479.8616069686643,
      "ter": 726
    },
    "secondary_primary_ratio": 0.0,
    "secondary_signal": 21.0,
    "secondary_signal_signed": 21.0,
    "tertiary_primary_ratio": 0.0,
    "tertiary_signal": 726.0,
    "tertiary_signal_signed": 726.0
  },
  "status": "ok"
}
Check Result
============================================================
ID          V11
Name        ModShift
Status      ok
Confidence  1.000

Metrics
------------------------------------------------------------
baseline_days            787.27
false_alarm_threshold    2.516974
fred                     54.286904
n_points                 73805
n_transits_expected      55
positive_primary_ratio   0.0
positive_signal          784.0
primary_signal           0.0
secondary_primary_ratio  0.0
secondary_signal         21.0
tertiary_primary_ratio   0.0
tertiary_signal          726.0
```

</details>


<details>
<summary><b>Analysis</b></summary>

- **Result:** ModShift metrics show **no detected secondary eclipse** at phase 0.5 (`secondary_signal = 21.0`) and a high `fred ≈ 54.29`.
- **Interpretation:** This is consistent with a **unique** transit-like event rather than a periodic eclipsing binary producing strong primary/secondary signatures.
- **Next check:** run **V11b** (independent uniqueness metrics) as a cross-check.

</details>


### V11b — ModShift uniqueness (independent)


In [ ]:
r = session.run('V11b')
print(json.dumps(r.model_dump(), indent=2, sort_keys=True))
print(btv.format_check_result(r, max_metrics=25))


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "confidence": 0.9,
  "flags": [],
  "id": "V11b",
  "metrics": {
    "chi": 3.1143362032152946,
    "fa1": 4.993861896556939,
    "fa2": 2.5169743319493008,
    "fred": 6.463155861663092,
    "med_chases": 0.11897041026036087,
    "ms1": -2.8498722240895185,
    "ms2": 7.624972278685888,
    "ms3": 5.895900447819491,
    "ms4": -4.378083604087844,
    "ms5": -2.2520960598105333,
    "ms6": -3.98116789067693,
    "n_in": 1941,
    "n_out": 71864,
    "n_transits": 10,
    "positive_primary_ratio": 0.39287641192527595,
    "secondary_primary_ratio": 0.2872114079544159,
    "sig_pos": 5.44406463918415,
    "sig_pri": 13.856939418952942,
    "sig_sec": 3.9798710804565207,
    "sig_ter": 3.7149928083177532,
    "tertiary_primary_ratio": 0.2680962004666443
  },
  "name": "ModShiftUniqueness",
  "notes": [],
  "provenance": {},
  "raw": {
    "chi": 3.1143362032152946,
    "fa1": 4.993861896556939,
    "fa2": 2.5169743319493008,
    "fred": 6.463155861663092,
    "med_chases": 0.11897041026036087,
    "ms1": -2.8498722240895185,
    "ms2": 7.624972278685888,
    "ms3": 5.895900447819491,
    "ms4": -4.378083604087844,
    "ms5": -2.2520960598105333,
    "ms6": -3.98116789067693,
    "n_in": 1941,
    "n_out": 71864,
    "n_transits": 10,
    "sig_pos": 5.44406463918415,
    "sig_pri": 13.856939418952942,
    "sig_sec": 3.9798710804565207,
    "sig_ter": 3.7149928083177532,
    "status": "ok",
    "warnings": []
  },
  "status": "ok"
}
Check Result
============================================================
ID          V11b
Name        ModShiftUniqueness
Status      ok
Confidence  0.900

Metrics
------------------------------------------------------------
chi                      3.1143362032152946
fa1                      4.993861896556939
fa2                      2.5169743319493008
fred                     6.463155861663092
med_chases               0.11897041026036087
ms1                      -2.8498722240895185
ms2                      7.624972278685888
ms3                      5.895900447819491
ms4                      -4.378083604087844
ms5                      -2.2520960598105333
ms6                      -3.98116789067693
n_in                     1941
n_out                    71864
n_transits               10
positive_primary_ratio   0.39287641192527595
secondary_primary_ratio  0.2872114079544159
sig_pos                  5.44406463918415
sig_pri                  13.856939418952942
sig_sec                  3.9798710804565207
sig_ter                  3.7149928083177532
tertiary_primary_ratio   0.2680962004666443
```

</details>


<details>
<summary><b>Analysis</b></summary>

- **Result:** `sig_pri ≈ 13.86` while `sig_sec ≈ 3.98` and `sig_ter ≈ 3.71`; ratios `secondary_primary_ratio ≈ 0.287` and `tertiary_primary_ratio ≈ 0.268`.
- **Interpretation:** The primary event is substantially stronger than secondary/tertiary events, supporting **signal uniqueness** (no strong EB-like secondary).
- **Next check:** run **V12 (SWEET)** to test whether **stellar variability** at the transit period could be masquerading as a transit.

</details>


### V12 — SWEET (variability at transit period)


In [ ]:
r = session.run('V12')
print(json.dumps(r.model_dump(), indent=2, sort_keys=True))
print(btv.format_check_result(r, max_metrics=25))


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "confidence": 1.0,
  "flags": [],
  "id": "V12",
  "metrics": {
    "baseline_days": 787.27,
    "cadence_median_min": 2.0,
    "n_points": 73805,
    "n_transits_expected": 55,
    "snr_at_period": 4.505338699040724,
    "snr_double_period": 0.37441144768396967,
    "snr_half_period": 4.3168120938420484,
    "sweet_msg": "WARN: SWEET test finds signal at HALF transit period\nWARN: SWEET test finds signal at the transit period"
  },
  "name": "SWEET",
  "notes": [],
  "provenance": {},
  "raw": {
    "_metrics_only": true,
    "inputs_summary": {
      "baseline_days": 787.27,
      "cadence_median_min": 2.0,
      "flux_err_available": true,
      "n_points": 73805,
      "n_transits_expected": 55
    },
    "raw_metrics": {
      "amp": [
        [
          5.867898263725434e-06,
          1.3593128762996233e-06,
          4.3168120938420484
        ],
        [
          6.124084129905695e-06,
          1.3592949473942179e-06,
          4.505338699040724
        ],
        [
          5.090070385524033e-07,
          1.3594857788163625e-06,
          0.37441144768396967
        ]
      ],
      "msg": "WARN: SWEET test finds signal at HALF transit period\nWARN: SWEET test finds signal at the transit period"
    },
    "snr_at_period": 4.505338699040724,
    "snr_double_period": 0.37441144768396967,
    "snr_half_period": 4.3168120938420484
  },
  "status": "ok"
}
Check Result
============================================================
ID          V12
Name        SWEET
Status      ok
Confidence  1.000

Metrics
------------------------------------------------------------
baseline_days        787.27
cadence_median_min   2.0
n_points             73805
n_transits_expected  55
snr_at_period        4.505338699040724
snr_double_period    0.37441144768396967
snr_half_period      4.3168120938420484
sweet_msg            WARN: SWEET test finds signal at HALF transit period
WARN: SWEET test finds signal at the transit period
```

</details>


<details>
<summary><b>Analysis</b></summary>
<ul>
<li><b>Result:</b> SWEET reports <code>snr_at_period ≈ 4.51</code> and <code>snr_half_period ≈ 4.32</code> (with warnings in <code>sweet_msg</code>).</li>
<li><b>Interpretation:</b> This is a <b>yellow flag</b>: variability near the transit period (and half-period) is detectable, so we should verify the transit is robust to <b>per-sector detrending/normalization</b> and confirm that the transit signature is not a sinusoidal/rotational harmonic artifact.</li>
<li><b>Next checks:</b> proceed to the LC-only false-alarm checks <b>V13 → V15</b>, and (ideally) re-run pixel/epoch diagnostics after a more robust per-sector detrend.</li>
</ul>
</details>


## 8) Light-curve false-alarm checks (V13, V15)

These are lightweight checks for common TESS false-alarm modes:
- **V13:** missing cadence / data gaps near predicted transits.
- **V15:** transit-window asymmetry (ramp/step-like systematics around transit windows).


### V13 — Data gaps near transits


In [ ]:
r = session.run('V13')
print(json.dumps(r.model_dump(), indent=2, sort_keys=True))
print(btv.format_check_result(r, max_metrics=25))


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "confidence": 0.75,
  "flags": [],
  "id": "V13",
  "metrics": {
    "missing_frac_max": 1.0,
    "missing_frac_median": 0.001,
    "n_epochs_evaluated": 10,
    "n_epochs_missing_ge_0p25": 2,
    "window_mult": 2.0
  },
  "name": "Data Gaps",
  "notes": [],
  "provenance": {},
  "raw": {
    "_metrics_only": true,
    "missing_frac_max": 1.0,
    "missing_frac_median": 0.001,
    "n_epochs_evaluated": 10,
    "n_epochs_missing_ge_0p25": 2,
    "window_mult": 2.0,
    "worst_epochs": [
      {
        "epoch_index": -50,
        "missing_frac": 1.0,
        "n_expected": 486,
        "n_observed": 0,
        "t_center_btjd": 2828.14455
      },
      {
        "epoch_index": -12,
        "missing_frac": 1.0,
        "n_expected": 486,
        "n_observed": 0,
        "t_center_btjd": 3369.3547012000004
      },
      {
        "epoch_index": -51,
        "missing_frac": 0.002057613168724326,
        "n_expected": 486,
        "n_observed": 485,
        "t_center_btjd": 2813.9021776
      },
      {
        "epoch_index": -14,
        "missing_frac": 0.002057613168724326,
        "n_expected": 486,
        "n_observed": 485,
        "t_center_btjd": 3340.8699564000003
      },
      {
        "epoch_index": 1,
        "missing_frac": 0.002057613168724326,
        "n_expected": 486,
        "n_observed": 485,
        "t_center_btjd": 3554.5055424
      },
      {
        "epoch_index": -52,
        "missing_frac": 0.0,
        "n_expected": 486,
        "n_observed": 486,
        "t_center_btjd": 2799.6598052
      },
      {
        "epoch_index": -13,
        "missing_frac": 0.0,
        "n_expected": 486,
        "n_observed": 486,
        "t_center_btjd": 3355.1123288000003
      },
      {
        "epoch_index": 0,
        "missing_frac": 0.0,
        "n_expected": 486,
        "n_observed": 486,
        "t_center_btjd": 3540.26317
      },
      {
        "epoch_index": 2,
        "missing_frac": 0.0,
        "n_expected": 486,
        "n_observed": 486,
        "t_center_btjd": 3568.7479148
      },
      {
        "epoch_index": 3,
        "missing_frac": 0.0,
        "n_expected": 486,
        "n_observed": 486,
        "t_center_btjd": 3582.9902872000002
      }
    ]
  },
  "status": "ok"
}
Check Result
============================================================
ID          V13
Name        Data Gaps
Status      ok
Confidence  0.750

Metrics
------------------------------------------------------------
missing_frac_max          1.0
missing_frac_median       0.001
n_epochs_evaluated        10
n_epochs_missing_ge_0p25  2
window_mult               2.0
```

</details>


<details>
<summary><b>Analysis</b></summary>

- **Result:** `n_epochs_evaluated = 10`; `n_epochs_missing_ge_0p25 = 2` and `missing_frac_max = 1.0`.
- **Interpretation:** Two predicted transit epochs land in windows with substantial missing data (including fully missing). This is common with multi-sector stitching and does not invalidate the candidate, but it means some epochs contribute **no information** and should be excluded from per-epoch consistency claims.
- **Next check:** run **V15** to look for ramp/step-like asymmetry around the transit windows.

</details>


### V15 — Transit-window asymmetry (ramp/step proxy)


In [ ]:
r = session.run('V15')
print(json.dumps(r.model_dump(), indent=2, sort_keys=True))
print(btv.format_check_result(r, max_metrics=25))


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "confidence": 0.75,
  "flags": [],
  "id": "V15",
  "metrics": {
    "asymmetry_sigma": 1.678,
    "baseline": 0.999969,
    "mu_left": -1.7e-05,
    "mu_right": -8.6e-05,
    "n_bins_half": 12,
    "n_left_bins": 12,
    "n_left_points": 1942,
    "n_right_bins": 12,
    "n_right_points": 1943,
    "window_mult": 2.0
  },
  "name": "Transit Asymmetry",
  "notes": [],
  "provenance": {},
  "raw": {
    "_metrics_only": true,
    "asymmetry_sigma": 1.678,
    "baseline": 0.999969,
    "mu_left": -1.7e-05,
    "mu_right": -8.6e-05,
    "n_bins_half": 12,
    "n_left_bins": 12,
    "n_left_points": 1942,
    "n_right_bins": 12,
    "n_right_points": 1943,
    "window_mult": 2.0
  },
  "status": "ok"
}
Check Result
============================================================
ID          V15
Name        Transit Asymmetry
Status      ok
Confidence  0.750

Metrics
------------------------------------------------------------
asymmetry_sigma  1.678
baseline         0.999969
mu_left          -1.7e-05
mu_right         -8.6e-05
n_bins_half      12
n_left_bins      12
n_left_points    1942
n_right_bins     12
n_right_points   1943
window_mult      2.0
```

</details>


<details>
<summary><b>Analysis</b></summary>

- **Result:** asymmetry is small (`asymmetry_sigma ≈ 1.68`) with left/right means `mu_left ≈ -1.7×10⁻⁵` and `mu_right ≈ -8.6×10⁻⁵`.
- **Interpretation:** No strong evidence for a ramp/step-like systematic that would preferentially distort one side of the transit window.
- **Next:** with LC-only checks complete, proceed to **AO-assisted FPP** and (given V09/V10/V12 caution) re-check robustness with **per-sector detrending/normalization**.

</details>


## 9) Extended diagnostics (V16–V19)

These checks are **diagnostic metrics** (not strict pass/fail gates). They are especially relevant here because:
- **V12 (SWEET)** detected variability power at the transit period and half-period.
- Pixel-level checks (**V09/V10**) looked fragile/mixed for this bright target.

We run them using the `preset='extended'` check registry.


In [ ]:
session_ext = btv.VettingSession.from_api(
    lc=btv.LightCurve(
        time=stitched.time,
        flux=stitched.flux,
        flux_err=stitched.flux_err,
        quality=stitched.quality,
    ),
    candidate=candidate,
    stellar=STELLAR,
    network=False,
    preset='extended',
    tic_id=TIC_ID,
)

print(json.dumps({'session_ext': 'ready', 'checks': ['V16', 'V17', 'V18', 'V19']}, indent=2))


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "session_ext": "ready",
  "checks": [
    "V16",
    "V17",
    "V18",
    "V19"
  ]
}
```

</details>


### V16 — Model competition (transit-only vs transit+sinusoid vs EB-like)


In [ ]:
r = session_ext.run('V16')
print(json.dumps(r.model_dump(), indent=2, sort_keys=True))
print(btv.format_check_result(r, max_metrics=25))


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "confidence": null,
  "flags": [
    "MODEL_PREFERS_NON_TRANSIT"
  ],
  "id": "V16",
  "metrics": {
    "aic_eb_like": -1001048.857269804,
    "aic_transit_only": -1001020.4102233441,
    "aic_transit_sinusoid": -1001078.4687633369,
    "artifact_prior_combined_risk": 0.2401026934306569,
    "artifact_prior_period_alias_risk": 0.4802053868613138,
    "artifact_prior_scattered_light_risk": 0.0,
    "artifact_prior_sector_quality_risk": 0.0,
    "artifact_risk": 0.8,
    "bic_eb_like": -1001021.2297245271,
    "bic_transit_only": -1001011.2010415852,
    "bic_transit_sinusoid": -1001032.422854542,
    "model_competition_label": "SINUSOID",
    "winner": "transit_sinusoid",
    "winner_margin_bic": 11.19313001492992
  },
  "name": "Model Competition",
  "notes": [
    "Sinusoidal variability model preferred - signal may be stellar rotation or pulsation"
  ],
  "provenance": {
    "alias_tolerance": 0.01,
    "bic_threshold": 10.0,
    "n_harmonics": 2
  },
  "raw": {
    "artifact_prior": {
      "combined_risk": 0.2401026934306569,
      "period_alias_risk": 0.4802053868613138,
      "scattered_light_risk": 0.0,
      "sector_quality_risk": 0.0
    },
    "fits": {
      "eb_like": {
        "aic": -1001048.857269804,
        "bic": -1001021.2297245271,
        "fitted_params": {
          "depth_avg": 0.00024893282565401244,
          "depth_avg_ppm": 248.93282565401245,
          "depth_even": 0.00028434192374280357,
          "depth_even_ppm": 284.34192374280354,
          "depth_odd": 0.00021352372756522128,
          "depth_odd_ppm": 213.5237275652213,
          "depth_secondary": -1.5075046995552246e-05,
          "depth_secondary_ppm": -15.075046995552245,
          "odd_even_diff_frac": 0.2844871743673183
        },
        "log_likelihood": 500527.428634902,
        "model_type": "eb_like",
        "n_params": 3,
        "residual_rms": 0.0002594680395204093
      },
      "transit_only": {
        "aic": -1001020.4102233441,
        "bic": -1001011.2010415852,
        "fitted_params": {
          "depth": 0.00024902725555213005,
          "depth_ppm": 249.02725555213004
        },
        "log_likelihood": 500511.20511167205,
        "model_type": "transit_only",
        "n_params": 1,
        "residual_rms": 0.00025950480882094704
      },
      "transit_sinusoid": {
        "aic": -1001078.4687633369,
        "bic": -1001032.422854542,
        "fitted_params": {
          "amplitude_k1": 6.5077277153130585e-06,
          "amplitude_k2": 6.2079809641080506e-06,
          "depth": 0.0002532809633687578,
          "depth_ppm": 253.2809633687578,
          "n_harmonics": 2,
          "phase_k1_rad": 1.3012730319138575,
          "phase_k2_rad": 1.877350756121778
        },
        "log_likelihood": 500544.23438166844,
        "model_type": "transit_sinusoid",
        "n_params": 5,
        "residual_rms": 0.00025943058128884264
      }
    },
    "warnings": [
      "Sinusoidal variability model preferred - signal may be stellar rotation or pulsation"
    ]
  },
  "status": "ok"
}
Check Result
============================================================
ID          V16
Name        Model Competition
Status      ok
Confidence

Metrics
------------------------------------------------------------
aic_eb_like                          -1001048.857269804
aic_transit_only                     -1001020.4102233441
aic_transit_sinusoid                 -1001078.4687633369
artifact_prior_combined_risk         0.2401026934306569
artifact_prior_period_alias_risk     0.4802053868613138
artifact_prior_scattered_light_risk  0.0
artifact_prior_sector_quality_risk   0.0
artifact_risk                        0.8
bic_eb_like                          -1001021.2297245271
bic_transit_only                     -1001011.2010415852
bic_transit_sinusoid                 -1001032.422854542
model_competition_label              SINUSOID
winner                               transit_sinusoid
winner_margin_bic                    11.19313001492992

Flags
------------------------------------------------------------
- MODEL_PREFERS_NON_TRANSIT

Notes
------------------------------------------------------------
- Sinusoidal variability model preferred - signal may be stellar rotation or pulsation
```

</details>


<details>
<summary><b>Analysis</b></summary>

<ul>
<li><b>Result:</b> the winning model is <code>transit_sinusoid</code> with <code>model_competition_label = "SINUSOID"</code> and <code>winner_margin_bic ≈ 11.19</code>; this triggers <code>MODEL_PREFERS_NON_TRANSIT</code> and an <code>artifact_risk = 0.8</code>.</li>
<li><b>Interpretation:</b> this is a real robustness warning: the stitched light curve is better explained by “transit + coherent sinusoid” than by “transit only”. Given the V12 SWEET warning, you should treat the signal as potentially coupled to stellar variability / systematics until it survives per-sector detrending/normalization.</li>
<li><b>Next check:</b> run <b>V17</b> (phase-shift null / ephemeris specificity) to quantify how special the claimed ephemeris is relative to phase-shifted nulls.</li>
</ul>

</details>


### V17 — Ephemeris reliability (phase-shift null)


In [ ]:
r = session_ext.run('V17')
print(json.dumps(r.model_dump(), indent=2, sort_keys=True))
print(btv.format_check_result(r, max_metrics=25))


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "confidence": null,
  "flags": [],
  "id": "V17",
  "metrics": {
    "depth_hat_ppm": 252.57172720708812,
    "depth_sigma_ppm": 6.72726577230079,
    "effective_n_points": 770.8631381129171,
    "max_ablation_score_drop_fraction": 0.018014316033769004,
    "n_in_transit": 972,
    "null_percentile": 0.9950248756218906,
    "period_neighborhood_best_period_days": 14.2423724,
    "period_neighborhood_best_score": 37.54448475144845,
    "period_neighborhood_second_best_score": 31.101255013267536,
    "period_peak_to_next_ratio": 1.2071694449446586,
    "phase_shift_null_p_value": 0.004975124378109453,
    "phase_shift_null_z": 6.1268661178199455,
    "score": 37.54448475144845,
    "top_5_fraction_abs": 0.0
  },
  "name": "Ephemeris Reliability Regime",
  "notes": [],
  "provenance": {
    "ingress_egress_fraction": 0.2,
    "sharpness": 30.0
  },
  "raw": {
    "ablation": [
      {
        "n_removed": 1,
        "score": 37.40314962321676,
        "score_drop_fraction": 0.003764471111199242
      },
      {
        "n_removed": 3,
        "score": 37.13248167447702,
        "score_drop_fraction": 0.01097373102065372
      },
      {
        "n_removed": 5,
        "score": 36.86814653781084,
        "score_drop_fraction": 0.018014316033769004
      }
    ],
    "base": {
      "depth_hat": 0.0002525717272070881,
      "depth_hat_ppm": 252.57172720708812,
      "depth_sigma": 6.72726577230079e-06,
      "depth_sigma_ppm": 6.72726577230079,
      "score": 37.54448475144845
    },
    "concentration": {
      "effective_n_points": 770.8631381129171,
      "in_transit_contribution_abs": 0.9914028516130647,
      "max_point_fraction_abs": 0.003589843357992594,
      "n_in_transit": 972,
      "top_5_fraction_abs": 0.017276918981830265
    },
    "harmonics": {
      "period_x0.5": {
        "period_days": 7.1211862,
        "score": 26.159820106230143
      },
      "period_x2": {
        "period_days": 28.4847448,
        "score": 30.278981237317613
      }
    },
    "label": "ok",
    "max_ablation_score_drop_fraction": 0.018014316033769004,
    "null_percentile": 0.9950248756218906,
    "period_neighborhood": {
      "best_period_days": 14.2423724,
      "best_score": 37.54448475144845,
      "peak_to_next": 1.2071694449446586,
      "period_grid_days": [
        14.2138876552,
        14.216736129680001,
        14.21958460416,
        14.222433078640002,
        14.22528155312,
        14.2281300276,
        14.230978502080001,
        14.23382697656,
        14.236675451040002,
        14.23952392552,
        14.2423724,
        14.245220874480001,
        14.24806934896,
        14.25091782344,
        14.253766297919999,
        14.256614772399999,
        14.259463246880001,
        14.262311721360001,
        14.265160195840002,
        14.26800867032,
        14.2708571448
      ],
      "score_at_input": 37.54448475144845,
      "scores": [
        21.377577629027115,
        23.501460699360678,
        24.048661582638957,
        25.81715688963544,
        24.09935937679657,
        22.728271146016326,
        23.909346757016976,
        25.86084330693275,
        28.842260449183783,
        31.101255013267536,
        37.54448475144845,
        26.937842755904104,
        20.910767851878546,
        19.21173084092568,
        17.411361570811938,
        15.949647577896926,
        18.02753809466936,
        16.910158633947432,
        17.298346845553496,
        15.54392015665596,
        14.700911115035822
      ],
      "second_best_score": 31.101255013267536
    },
    "phase_shift_null": {
      "n_trials": 200,
      "null_mean": -0.5359135769720348,
      "null_std": 6.215314256282494,
      "p_value_one_sided": 0.004975124378109453,
      "strategy": "grid",
      "z_score": 6.1268661178199455
    },
    "t0_sensitivity": {
      "backend": "numpy",
      "curvature": -15127.809618013054,
      "delta_score": 0.1671115649641024,
      "fwhm_minutes": 200.2769999998418,
      "half_span_minutes": 364.14000000000004,
      "n_grid": 81,
      "score_at_input": 37.54448475144845,
      "score_best": 37.71159631641255,
      "t0_best_btjd": 3540.2694918750003
    },
    "top_contribution_fractions": {
      "top_10_fraction": 0.032511782717225025,
      "top_1_fraction": 0.003589843357992594,
      "top_3_fraction": 0.01050617516896018,
      "top_5_fraction": 0.017276918981830265
    },
    "warnings": []
  },
  "status": "ok"
}
Check Result
============================================================
ID          V17
Name        Ephemeris Reliability Regime
Status      ok
Confidence

Metrics
------------------------------------------------------------
depth_hat_ppm                          252.57172720708812
depth_sigma_ppm                        6.72726577230079
effective_n_points                     770.8631381129171
max_ablation_score_drop_fraction       0.018014316033769004
n_in_transit                           972
null_percentile                        0.9950248756218906
period_neighborhood_best_period_days   14.2423724
period_neighborhood_best_score         37.54448475144845
period_neighborhood_second_best_score  31.101255013267536
period_peak_to_next_ratio              1.2071694449446586
phase_shift_null_p_value               0.004975124378109453
phase_shift_null_z                     6.1268661178199455
score                                  37.54448475144845
top_5_fraction_abs                     0.0
```

</details>


<details>
<summary><b>Analysis</b></summary>

<ul>
<li><b>Result:</b> <code>phase_shift_null_p_value ≈ 0.00498</code> with <code>phase_shift_null_z ≈ 6.13</code>, and <code>null_percentile ≈ 0.995</code>. The base detection has <code>score ≈ 37.54</code> and <code>depth_hat_ppm ≈ 252.57</code> with <code>depth_sigma_ppm ≈ 6.73</code>.</li>
<li><b>Interpretation:</b> relative to phase-shifted nulls, the ephemeris is quite “special” (low p-value / high z). This supports the claim that the signal is not easily replicated by arbitrary phase shifts, but it does <i>not</i> override the V16/V12 variability concern—those speak to model degeneracy with sinusoids.</li>
<li><b>Next check:</b> run <b>V18</b> (sensitivity sweep; optional and may be skipped if the backend isn’t available).</li>
</ul>

</details>


### V18 — Sensitivity sweep (optional)


In [ ]:
r = session_ext.run('V18')
print(json.dumps(r.model_dump(), indent=2, sort_keys=True))
print(btv.format_check_result(r, max_metrics=25))


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "confidence": null,
  "flags": [
    "SKIPPED:EXTRA_MISSING:celerite2"
  ],
  "id": "V18",
  "metrics": {},
  "name": "Ephemeris Sensitivity Sweep",
  "notes": [
    "Check skipped: missing optional dependency 'celerite2'. Install with: pip install 'bittr-tess-vetter[celerite2]'"
  ],
  "provenance": {},
  "raw": null,
  "status": "skipped"
}
Check Result
============================================================
ID          V18
Name        Ephemeris Sensitivity Sweep
Status      skipped
Confidence

Flags
------------------------------------------------------------
- SKIPPED:EXTRA_MISSING:celerite2

Notes
------------------------------------------------------------
- Check skipped: missing optional dependency 'celerite2'. Install with: pip install 'bittr-tess-vetter[celerite2]'
```

</details>


<details>
<summary><b>Analysis</b></summary>

<ul>
<li><b>Result:</b> this check is <code>skipped</code> because the optional dependency <code>celerite2</code> is not installed (<code>SKIPPED:EXTRA_MISSING:celerite2</code>).</li>
<li><b>Interpretation:</b> nothing is “failing” here—this is an optional advanced diagnostic. If you want to enable it, install <code>bittr-tess-vetter[celerite2]</code> and re-run.</li>
<li><b>Next check:</b> run <b>V19</b> (alias/harmonic + phase-shift event diagnostics).</li>
</ul>

</details>


### V19 — Alias/harmonic diagnostics + phase-shift events


In [ ]:
r = session_ext.run('V19')
print(json.dumps(r.model_dump(), indent=2, sort_keys=True))
print(btv.format_check_result(r, max_metrics=25))


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "confidence": null,
  "flags": [],
  "id": "V19",
  "metrics": {
    "base_depth_ppm_P": 255.9925026659471,
    "base_score_P": 37.6479248900702,
    "best_other_depth_ppm": 216.87980325213462,
    "best_other_harmonic": "2P",
    "best_other_over_base_score_ratio": 0.7162470784918131,
    "best_other_score": 26.965216213791997,
    "max_phase_shift_event_sigma": 16.09151334844518,
    "n_phase_shift_events": 5,
    "secondary_significance_sigma": 0.0
  },
  "name": "Alias Diagnostics",
  "notes": [],
  "provenance": {
    "event_sigma_threshold": 3.0,
    "n_phase_bins": 10
  },
  "raw": {
    "harmonic_scores": [
      {
        "depth_ppm": 255.9925026659471,
        "duration_hours": 4.046,
        "harmonic": "P",
        "period": 14.2423724,
        "score": 37.6479248900702
      },
      {
        "depth_ppm": 132.25985085529857,
        "duration_hours": 2.8609540366807718,
        "harmonic": "P/2",
        "period": 7.1211862,
        "score": 22.141245419639674
      },
      {
        "depth_ppm": 216.87980325213462,
        "duration_hours": 5.7219080733615435,
        "harmonic": "2P",
        "period": 28.4847448,
        "score": 26.965216213791997
      },
      {
        "depth_ppm": 112.69060087082394,
        "duration_hours": 2.3359591891412257,
        "harmonic": "P/3",
        "period": 4.747457466666667,
        "score": 20.867438494185677
      },
      {
        "depth_ppm": 93.09457177086867,
        "duration_hours": 7.0078775674236775,
        "harmonic": "3P",
        "period": 42.7271172,
        "score": 11.136728857593493
      }
    ],
    "phase_shift_events": [
      {
        "depth_ppm": 46.96190271624712,
        "n_points": 8178,
        "phase": 0.05,
        "significance": 16.09151334844518
      },
      {
        "depth_ppm": 14.481505519192694,
        "n_points": 7005,
        "phase": 0.25,
        "significance": 4.592460568876179
      },
      {
        "depth_ppm": 24.55667847411913,
        "n_points": 7136,
        "phase": 0.45,
        "significance": 7.860038925722026
      },
      {
        "depth_ppm": 17.19483779227815,
        "n_points": 8044,
        "phase": 0.6500000000000001,
        "significance": 5.843348201286424
      },
      {
        "depth_ppm": 28.385192090363276,
        "n_points": 7408,
        "phase": 0.8500000000000001,
        "significance": 9.256993879522128
      }
    ]
  },
  "status": "ok"
}
Check Result
============================================================
ID          V19
Name        Alias Diagnostics
Status      ok
Confidence

Metrics
------------------------------------------------------------
base_depth_ppm_P                  255.9925026659471
base_score_P                      37.6479248900702
best_other_depth_ppm              216.87980325213462
best_other_harmonic               2P
best_other_over_base_score_ratio  0.7162470784918131
best_other_score                  26.965216213791997
max_phase_shift_event_sigma       16.09151334844518
n_phase_shift_events              5
secondary_significance_sigma      0.0
```

</details>


<details>
<summary><b>Analysis</b></summary>

<ul>
<li><b>Result:</b> the best competing harmonic is <code>2P</code> with <code>best_other_over_base_score_ratio ≈ 0.716</code> (base <code>score_P ≈ 37.65</code>, other <code>score ≈ 26.97</code>). It also reports <code>n_phase_shift_events = 5</code> with <code>max_phase_shift_event_sigma ≈ 16.09</code>.</li>
<li><b>Interpretation:</b> the harmonic competition is not equal (2P is weaker), but the presence of multiple high-significance phase-shift events is a strong prompt for human inspection and per-sector robustness work (these can indicate systematics/aliasing in real TESS data).</li>
<li><b>Next:</b> implement per-sector detrending/normalization and re-run the sensitive diagnostics (<b>V12 → V16 → V17 → V19</b>). If those remain favorable, proceed to AO-assisted FPP.</li>
</ul>

</details>


## 10) Per-sector detrending/normalization (robustness follow-up)

Because **V12** (SWEET) and **V16** (model competition) indicate variability risk, we now re-run the sensitive checks after:
- detrending each sector *with transits masked* (to avoid distorting the transit)
- normalizing each sector to a common baseline
- stitching the detrended sectors

Goal: determine whether the “sinusoid preference” collapses under a more robust per-sector treatment.


In [ ]:
DETREND_METHOD = 'wotan_biweight'
WOTAN_WINDOW_DAYS = 0.5

detrended_by_sector: dict[int, btv.LightCurve] = {}
summary = []

for sector in sorted(ds.lc_by_sector.keys()):
    sector = int(sector)
    sec = ds.lc_by_sector[sector]

    lc_sec = btv.LightCurve(time=sec.time, flux=sec.flux, flux_err=sec.flux_err)

    # Build a transit mask (avoid fitting away the transit while detrending)
    duration_days = candidate.ephemeris.duration_hours / 24.0
    phase = ((np.asarray(lc_sec.time) - candidate.ephemeris.t0_btjd) / candidate.ephemeris.period_days) % 1.0
    transit_phase_width = 1.5 * duration_days / candidate.ephemeris.period_days
    transit_mask = (phase < transit_phase_width) | (phase > (1.0 - transit_phase_width))

    flat_flux = btv.wotan_flatten(
        time=np.asarray(lc_sec.time, dtype=np.float64),
        flux=np.asarray(lc_sec.flux, dtype=np.float64),
        window_length=WOTAN_WINDOW_DAYS,
        method='biweight',
        transit_mask=np.asarray(transit_mask, dtype=bool),
    )

    norm_flux, norm_err = btv.normalize_flux(
        np.asarray(flat_flux, dtype=np.float64),
        np.asarray(lc_sec.flux_err, dtype=np.float64),
    )
    flat_norm = btv.LightCurve(time=lc_sec.time, flux=norm_flux, flux_err=norm_err)
    detrended_by_sector[sector] = flat_norm

    summary.append(
        {
            'sector': sector,
            'n_points': int(len(flat_norm.time)),
            'flux_median': float(np.nanmedian(flat_norm.flux)),
            'flux_std': float(np.nanstd(flat_norm.flux)),
        }
    )

print(
    json.dumps(
        {
            'detrend_method': DETREND_METHOD,
            'window_days': WOTAN_WINDOW_DAYS,
            'sectors': sorted(detrended_by_sector.keys()),
            'per_sector_summary': summary,
        },
        indent=2,
        sort_keys=True,
    )
)


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "detrend_method": "wotan_biweight",
  "per_sector_summary": [
    {
      "flux_median": 1.0,
      "flux_std": 0.0002668133149133553,
      "n_points": 18876,
      "sector": 55
    },
    {
      "flux_median": 1.0,
      "flux_std": 0.0002551851488528821,
      "n_points": 19470,
      "sector": 75
    },
    {
      "flux_median": 1.0,
      "flux_std": 0.00024613464683548346,
      "n_points": 18133,
      "sector": 82
    },
    {
      "flux_median": 1.0,
      "flux_std": 0.0002349506399182097,
      "n_points": 17326,
      "sector": 83
    }
  ],
  "sectors": [
    55,
    75,
    82,
    83
  ],
  "window_days": 0.5
}
```

</details>


In [ ]:
stitched_det = btv.stitch_lightcurves(
    [
        {
            'time': np.asarray(lc.time, dtype=np.float64),
            'flux': np.asarray(lc.flux, dtype=np.float64),
            'flux_err': np.asarray(lc.flux_err, dtype=np.float64),
            'sector': int(sector),
            'quality': np.zeros(len(lc.time), dtype=np.int32),
        }
        for sector, lc in sorted(detrended_by_sector.items())
    ]
)

in_tr = btv.get_in_transit_mask(stitched_det.time, PERIOD_DAYS, T0_BTJD, DURATION_HOURS)
out_tr = btv.get_out_of_transit_mask(
    stitched_det.time,
    PERIOD_DAYS,
    T0_BTJD,
    DURATION_HOURS,
    buffer_factor=3.0,
)
depth_hat_det, depth_err_det = btv.measure_transit_depth(stitched_det.flux, in_tr, out_tr)
DEPTH_PPM_DET = float(depth_hat_det * 1e6)
DEPTH_ERR_PPM_DET = float(depth_err_det * 1e6)

candidate_det = btv.Candidate(ephemeris=ephem, depth_ppm=DEPTH_PPM_DET)

print(
    json.dumps(
        {
            'depth_ppm_hat_detrended': DEPTH_PPM_DET,
            'depth_ppm_err_detrended': DEPTH_ERR_PPM_DET,
            'n_points_stitched_detrended': int(len(stitched_det.time)),
        },
        indent=2,
        sort_keys=True,
    )
)


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "depth_ppm_err_detrended": 8.401335187911371,
  "depth_ppm_hat_detrended": 240.46223372842235,
  "n_points_stitched_detrended": 73805
}
```

</details>


### Re-run sensitive diagnostics on the detrended light curve


In [ ]:
session_det = btv.VettingSession.from_api(
    lc=btv.LightCurve(
        time=stitched_det.time,
        flux=stitched_det.flux,
        flux_err=stitched_det.flux_err,
        quality=stitched_det.quality,
    ),
    candidate=candidate_det,
    stellar=STELLAR,
    network=False,
    preset='default',
    tic_id=TIC_ID,
)

session_ext_det = btv.VettingSession.from_api(
    lc=btv.LightCurve(
        time=stitched_det.time,
        flux=stitched_det.flux,
        flux_err=stitched_det.flux_err,
        quality=stitched_det.quality,
    ),
    candidate=candidate_det,
    stellar=STELLAR,
    network=False,
    preset='extended',
    tic_id=TIC_ID,
)

print(json.dumps({'session_det': 'ready', 'session_ext_det': 'ready'}, indent=2, sort_keys=True))


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "session_det": "ready",
  "session_ext_det": "ready"
}
```

</details>


#### V12 (detrended) — SWEET


In [ ]:
r = session_det.run('V12')
print(json.dumps(r.model_dump(), indent=2, sort_keys=True))
print(btv.format_check_result(r, max_metrics=25))


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "confidence": 1.0,
  "flags": [],
  "id": "V12",
  "metrics": {
    "baseline_days": 787.27,
    "cadence_median_min": 2.0,
    "n_points": 73805,
    "n_transits_expected": 55,
    "snr_at_period": 1.08801880004192,
    "snr_double_period": 0.1951146014429856,
    "snr_half_period": 0.12811380949306134,
    "sweet_msg": "OK: SWEET finds no out-of-transit variability at transit period"
  },
  "name": "SWEET",
  "notes": [],
  "provenance": {},
  "raw": {
    "_metrics_only": true,
    "inputs_summary": {
      "baseline_days": 787.27,
      "cadence_median_min": 2.0,
      "flux_err_available": true,
      "n_points": 73805,
      "n_transits_expected": 55
    },
    "raw_metrics": {
      "amp": [
        [
          1.6774451958607758e-07,
          1.309339877175088e-06,
          0.12811380949306134
        ],
        [
          1.4245748924000321e-06,
          1.3093292986712593e-06,
          1.08801880004192
        ],
        [
          2.554712909967999e-07,
          1.30933968604831e-06,
          0.1951146014429856
        ]
      ],
      "msg": "OK: SWEET finds no out-of-transit variability at transit period"
    },
    "snr_at_period": 1.08801880004192,
    "snr_double_period": 0.1951146014429856,
    "snr_half_period": 0.12811380949306134
  },
  "status": "ok"
}
Check Result
============================================================
ID          V12
Name        SWEET
Status      ok
Confidence  1.000

Metrics
------------------------------------------------------------
baseline_days        787.27
cadence_median_min   2.0
n_points             73805
n_transits_expected  55
snr_at_period        1.08801880004192
snr_double_period    0.1951146014429856
snr_half_period      0.12811380949306134
sweet_msg            OK: SWEET finds no out-of-transit variability at transit period
```

</details>


<details>
<summary><b>Analysis</b></summary>

<ul>
<li><b>Result:</b> SWEET collapses to <code>snr_at_period ≈ 1.09</code> and <code>snr_half_period ≈ 0.13</code> with <code>sweet_msg</code> = “OK: SWEET finds no out-of-transit variability at transit period”.</li>
<li><b>Interpretation:</b> this strongly suggests the earlier SWEET warning was driven by sector-level trends / variability that can be removed with transit-masked, per-sector detrending.</li>
<li><b>Next:</b> re-run <b>V16</b> to see whether the sinusoid-preference also collapses.</li>
</ul>

</details>


#### V16 (detrended) — Model competition


In [ ]:
r = session_ext_det.run('V16')
print(json.dumps(r.model_dump(), indent=2, sort_keys=True))
print(btv.format_check_result(r, max_metrics=25))


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "confidence": null,
  "flags": [],
  "id": "V16",
  "metrics": {
    "aic_eb_like": -1009237.1737519107,
    "aic_transit_only": -1009233.0212192924,
    "aic_transit_sinusoid": -1009227.0578637859,
    "artifact_prior_combined_risk": 0.2401026934306569,
    "artifact_prior_period_alias_risk": 0.4802053868613138,
    "artifact_prior_scattered_light_risk": 0.0,
    "artifact_prior_sector_quality_risk": 0.0,
    "artifact_risk": 0.0,
    "bic_eb_like": -1009209.5462066338,
    "bic_transit_only": -1009223.8120375335,
    "bic_transit_sinusoid": -1009181.011954991,
    "model_competition_label": "TRANSIT",
    "winner": "transit_only",
    "winner_margin_bic": 14.265830899705179
  },
  "name": "Model Competition",
  "notes": [],
  "provenance": {
    "alias_tolerance": 0.01,
    "bic_threshold": 10.0,
    "n_harmonics": 2
  },
  "raw": {
    "artifact_prior": {
      "combined_risk": 0.2401026934306569,
      "period_alias_risk": 0.4802053868613138,
      "scattered_light_risk": 0.0,
      "sector_quality_risk": 0.0
    },
    "fits": {
      "eb_like": {
        "aic": -1009237.1737519107,
        "bic": -1009209.5462066338,
        "fitted_params": {
          "depth_avg": 0.0002326806550531314,
          "depth_avg_ppm": 232.6806550531314,
          "depth_even": 0.0002396930375240259,
          "depth_even_ppm": 239.6930375240259,
          "depth_odd": 0.00022566827258223691,
          "depth_odd_ppm": 225.6682725822369,
          "depth_secondary": -1.92139498425781e-05,
          "depth_secondary_ppm": -19.2139498425781,
          "odd_even_diff_frac": 0.060274735510721794
        },
        "log_likelihood": 504621.58687595534,
        "model_type": "eb_like",
        "n_params": 3,
        "residual_rms": 0.0002500220567807729
      },
      "transit_only": {
        "aic": -1009233.0212192924,
        "bic": -1009223.8120375335,
        "fitted_params": {
          "depth": 0.0002326993558766794,
          "depth_ppm": 232.6993558766794
        },
        "log_likelihood": 504617.5106096462,
        "model_type": "transit_only",
        "n_params": 1,
        "residual_rms": 0.0002500314894896405
      },
      "transit_sinusoid": {
        "aic": -1009227.0578637859,
        "bic": -1009181.011954991,
        "fitted_params": {
          "amplitude_k1": 1.507635459180498e-06,
          "amplitude_k2": 3.3915040098204686e-07,
          "depth": 0.0002321312191124991,
          "depth_ppm": 232.13121911249908,
          "n_harmonics": 2,
          "phase_k1_rad": 3.079169486651852,
          "phase_k2_rad": 2.258826501189109
        },
        "log_likelihood": 504618.52893189294,
        "model_type": "transit_sinusoid",
        "n_params": 5,
        "residual_rms": 0.00025002914312377095
      }
    },
    "warnings": []
  },
  "status": "ok"
}
Check Result
============================================================
ID          V16
Name        Model Competition
Status      ok
Confidence

Metrics
------------------------------------------------------------
aic_eb_like                          -1009237.1737519107
aic_transit_only                     -1009233.0212192924
aic_transit_sinusoid                 -1009227.0578637859
artifact_prior_combined_risk         0.2401026934306569
artifact_prior_period_alias_risk     0.4802053868613138
artifact_prior_scattered_light_risk  0.0
artifact_prior_sector_quality_risk   0.0
artifact_risk                        0.0
bic_eb_like                          -1009209.5462066338
bic_transit_only                     -1009223.8120375335
bic_transit_sinusoid                 -1009181.011954991
model_competition_label              TRANSIT
winner                               transit_only
winner_margin_bic                    14.265830899705179
```

</details>


<details>
<summary><b>Analysis</b></summary>

<ul>
<li><b>Result:</b> the winner flips to <code>transit_only</code> with <code>model_competition_label = "TRANSIT"</code> and <code>winner_margin_bic ≈ 14.27</code>. The previous <code>MODEL_PREFERS_NON_TRANSIT</code> flag disappears and <code>artifact_risk = 0.0</code>.</li>
<li><b>Interpretation:</b> this is exactly the robustness behavior we wanted: after per-sector detrending/normalization, the data no longer prefers “transit + coherent sinusoid”. That substantially reduces the “not a planet” concern raised by V16.</li>
<li><b>Next:</b> run <b>V17</b> and <b>V19</b> again to see whether the ephemeris specificity and phase-shift event picture also improves.</li>
</ul>

</details>


#### V17 (detrended) — Phase-shift null


In [ ]:
r = session_ext_det.run('V17')
print(json.dumps(r.model_dump(), indent=2, sort_keys=True))
print(btv.format_check_result(r, max_metrics=25))


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "confidence": null,
  "flags": [],
  "id": "V17",
  "metrics": {
    "depth_hat_ppm": 236.12225832964558,
    "depth_sigma_ppm": 6.7272829121944895,
    "effective_n_points": 766.9865056364969,
    "max_ablation_score_drop_fraction": 0.01852294751495275,
    "n_in_transit": 972,
    "null_percentile": 0.9950248756218906,
    "period_neighborhood_best_period_days": 14.2423724,
    "period_neighborhood_best_score": 35.099201477260415,
    "period_neighborhood_second_best_score": 26.647688735283015,
    "period_peak_to_next_ratio": 1.3171574400292032,
    "phase_shift_null_p_value": 0.004975124378109453,
    "phase_shift_null_z": 11.672078951720913,
    "score": 35.099201477260415,
    "top_5_fraction_abs": 0.0
  },
  "name": "Ephemeris Reliability Regime",
  "notes": [],
  "provenance": {
    "ingress_egress_fraction": 0.2,
    "sharpness": 30.0
  },
  "raw": {
    "ablation": [
      {
        "n_removed": 1,
        "score": 34.96380616417332,
        "score_drop_fraction": 0.0038575040852371783
      },
      {
        "n_removed": 3,
        "score": 34.70343009078539,
        "score_drop_fraction": 0.011275794599812561
      },
      {
        "n_removed": 5,
        "score": 34.44906081048037,
        "score_drop_fraction": 0.01852294751495275
      }
    ],
    "base": {
      "depth_hat": 0.00023612225832964558,
      "depth_hat_ppm": 236.12225832964558,
      "depth_sigma": 6.7272829121944896e-06,
      "depth_sigma_ppm": 6.7272829121944895,
      "score": 35.099201477260415
    },
    "concentration": {
      "effective_n_points": 766.9865056364969,
      "in_transit_contribution_abs": 0.9912432195527296,
      "max_point_fraction_abs": 0.0035722207600605844,
      "n_in_transit": 972,
      "top_5_fraction_abs": 0.01717928823639797
    },
    "harmonics": {
      "period_x0.5": {
        "period_days": 7.1211862,
        "score": 23.96530199431358
      },
      "period_x2": {
        "period_days": 28.4847448,
        "score": 25.510823279216122
      }
    },
    "label": "ok",
    "max_ablation_score_drop_fraction": 0.01852294751495275,
    "null_percentile": 0.9950248756218906,
    "period_neighborhood": {
      "best_period_days": 14.2423724,
      "best_score": 35.099201477260415,
      "peak_to_next": 1.3171574400292032,
      "period_grid_days": [
        14.2138876552,
        14.216736129680001,
        14.21958460416,
        14.222433078640002,
        14.22528155312,
        14.2281300276,
        14.230978502080001,
        14.23382697656,
        14.236675451040002,
        14.23952392552,
        14.2423724,
        14.245220874480001,
        14.24806934896,
        14.25091782344,
        14.253766297919999,
        14.256614772399999,
        14.259463246880001,
        14.262311721360001,
        14.265160195840002,
        14.26800867032,
        14.2708571448
      ],
      "score_at_input": 35.099201477260415,
      "scores": [
        15.247193327525629,
        16.113548335361074,
        15.445203897556137,
        16.825155134282408,
        15.488921959476968,
        15.007305967665173,
        16.957333996512663,
        18.91652289256253,
        22.443911460969044,
        25.49907214739846,
        35.099201477260415,
        26.647688735283015,
        20.142719544568337,
        17.903126596569674,
        15.152930919968826,
        12.599076753388179,
        14.014632631413187,
        12.091987768762182,
        12.506019349654506,
        10.594556151191169,
        9.082957820901381
      ],
      "second_best_score": 26.647688735283015
    },
    "phase_shift_null": {
      "n_trials": 200,
      "null_mean": -0.3399346393118668,
      "null_std": 3.036231699867588,
      "p_value_one_sided": 0.004975124378109453,
      "strategy": "grid",
      "z_score": 11.672078951720913
    },
    "t0_sensitivity": {
      "backend": "numpy",
      "curvature": -13476.644803271594,
      "delta_score": 0.22019733809273134,
      "fwhm_minutes": 218.48400000024412,
      "half_span_minutes": 364.14000000000004,
      "n_grid": 81,
      "score_at_input": 35.099201477260415,
      "score_best": 35.319398815353146,
      "t0_best_btjd": 3540.256848125
    },
    "top_contribution_fractions": {
      "top_10_fraction": 0.03276280173119653,
      "top_1_fraction": 0.0035722207600605844,
      "top_3_fraction": 0.010452736075857526,
      "top_5_fraction": 0.01717928823639797
    },
    "warnings": []
  },
  "status": "ok"
}
Check Result
============================================================
ID          V17
Name        Ephemeris Reliability Regime
Status      ok
Confidence

Metrics
------------------------------------------------------------
depth_hat_ppm                          236.12225832964558
depth_sigma_ppm                        6.7272829121944895
effective_n_points                     766.9865056364969
max_ablation_score_drop_fraction       0.01852294751495275
n_in_transit                           972
null_percentile                        0.9950248756218906
period_neighborhood_best_period_days   14.2423724
period_neighborhood_best_score         35.099201477260415
period_neighborhood_second_best_score  26.647688735283015
period_peak_to_next_ratio              1.3171574400292032
phase_shift_null_p_value               0.004975124378109453
phase_shift_null_z                     11.672078951720913
score                                  35.099201477260415
top_5_fraction_abs                     0.0
```

</details>


<details>
<summary><b>Analysis</b></summary>

<ul>
<li><b>Result:</b> the phase-shift null remains strongly favorable (<code>phase_shift_null_p_value ≈ 0.00498</code>; <code>null_percentile ≈ 0.995</code>), with <code>phase_shift_null_z ≈ 11.67</code> and <code>score ≈ 35.10</code> on the detrended series.</li>
<li><b>Interpretation:</b> the ephemeris remains “special” relative to phase-shifted nulls even after detrending, which is what we want to see if this is a real periodic transit signal.</li>
<li><b>Next:</b> run <b>V19</b> (detrended) and check whether the phase-shift event count/significance decreases.</li>
</ul>

</details>


#### V19 (detrended) — Alias + phase-shift events


In [ ]:
r = session_ext_det.run('V19')
print(json.dumps(r.model_dump(), indent=2, sort_keys=True))
print(btv.format_check_result(r, max_metrics=25))


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "confidence": null,
  "flags": [],
  "id": "V19",
  "metrics": {
    "base_depth_ppm_P": 236.87261783555957,
    "base_score_P": 34.83594157271188,
    "best_other_depth_ppm": 175.3531654834495,
    "best_other_harmonic": "2P",
    "best_other_over_base_score_ratio": 0.6258492683134627,
    "best_other_score": 21.802048544292266,
    "max_phase_shift_event_sigma": 6.384853282826452,
    "n_phase_shift_events": 2,
    "secondary_significance_sigma": 0.0
  },
  "name": "Alias Diagnostics",
  "notes": [],
  "provenance": {
    "event_sigma_threshold": 3.0,
    "n_phase_bins": 10
  },
  "raw": {
    "harmonic_scores": [
      {
        "depth_ppm": 236.87261783555957,
        "duration_hours": 4.046,
        "harmonic": "P",
        "period": 14.2423724,
        "score": 34.83594157271188
      },
      {
        "depth_ppm": 125.0883199667152,
        "duration_hours": 2.8609540366807718,
        "harmonic": "P/2",
        "period": 7.1211862,
        "score": 20.940626357704602
      },
      {
        "depth_ppm": 175.3531654834495,
        "duration_hours": 5.7219080733615435,
        "harmonic": "2P",
        "period": 28.4847448,
        "score": 21.802048544292266
      },
      {
        "depth_ppm": 98.76528158203612,
        "duration_hours": 2.3359591891412257,
        "harmonic": "P/3",
        "period": 4.747457466666667,
        "score": 18.28877793220467
      },
      {
        "depth_ppm": 118.32023867386354,
        "duration_hours": 7.0078775674236775,
        "harmonic": "3P",
        "period": 42.7271172,
        "score": 14.154388556256391
      }
    ],
    "phase_shift_events": [
      {
        "depth_ppm": 11.550485902844798,
        "n_points": 8178,
        "phase": 0.05,
        "significance": 4.109152059004758
      },
      {
        "depth_ppm": 18.442160866105617,
        "n_points": 7745,
        "phase": 0.95,
        "significance": 6.384853282826452
      }
    ]
  },
  "status": "ok"
}
Check Result
============================================================
ID          V19
Name        Alias Diagnostics
Status      ok
Confidence

Metrics
------------------------------------------------------------
base_depth_ppm_P                  236.87261783555957
base_score_P                      34.83594157271188
best_other_depth_ppm              175.3531654834495
best_other_harmonic               2P
best_other_over_base_score_ratio  0.6258492683134627
best_other_score                  21.802048544292266
max_phase_shift_event_sigma       6.384853282826452
n_phase_shift_events              2
secondary_significance_sigma      0.0
```

</details>


<details>
<summary><b>Analysis</b></summary>

<ul>
<li><b>Result:</b> <code>n_phase_shift_events</code> drops to <b>2</b> with <code>max_phase_shift_event_sigma ≈ 6.38</code> (previously 5 events with max sigma ≈ 16.09). The best competing harmonic remains <code>2P</code> but is weaker (<code>best_other_over_base_score_ratio ≈ 0.626</code>).</li>
<li><b>Interpretation:</b> detrending reduces the number and severity of off-ephemeris “events”, which is consistent with those being driven by residual variability/systematics in the undetrended stitched series.</li>
<li><b>Next:</b> with V12/V16/V19 improved under per-sector detrending, proceed to AO-assisted FPP and (optionally) a per-sector re-check of pixel localization on the detrended series.</li>
</ul>

</details>


## 11) AO-assisted statistical validation (TRICERATOPS FPP/NFPP)

We now compute a TRICERATOPS(+)-style false positive probability using the **PHARO AO contrast curve** bundled with this tutorial dataset.

Notes:
- This is a **statistical** validation step. It uses nearby-source constraints (Gaia) and high-resolution imaging limits to reduce blend/EB probability.
- We intentionally cache the per-sector light curves on disk (same pattern as tutorial 04) so the run is reproducible.


In [ ]:
FPP_PRESET = 'fast'  # switch to 'standard' for a higher-fidelity (slower) run
FPP_SEED = 123
FPP_REPLICATES = 1
FPP_CACHE_DIR = 'persistent_cache/fpp_cache'

contrast_curve_path = ds.root / 'PHARO_Kcont_plot.tbl'
contrast_curve = btv.load_contrast_curve_exofop_tbl(contrast_curve_path, filter='K')

cache = btv.hydrate_cache_from_dataset(dataset=ds, tic_id=TIC_ID, cache_dir=FPP_CACHE_DIR)

print(
    json.dumps(
        {
            'contrast_curve_path': str(contrast_curve_path),
            'contrast_curve_filter': contrast_curve.filter,
            'contrast_curve_n_points': int(len(contrast_curve.separation_arcsec)),
            'fpp_preset': FPP_PRESET,
            'fpp_seed': FPP_SEED,
            'fpp_replicates': FPP_REPLICATES,
            'fpp_cache_dir': str(FPP_CACHE_DIR),
        },
        indent=2,
        sort_keys=True,
    )
)


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "contrast_curve_filter": "K",
  "contrast_curve_n_points": 97,
  "contrast_curve_path": "/Users/collier/projects/apps/bittr-tess-vetter/docs/tutorials/data/tic188646744/PHARO_Kcont_plot.tbl",
  "fpp_cache_dir": "persistent_cache/fpp_cache",
  "fpp_preset": "fast",
  "fpp_replicates": 1,
  "fpp_seed": 123
}
```

</details>


In [ ]:
import contextlib
import io
import warnings

# TRICERATOPS prints a lot of informational text + warnings; capture stdout/stderr so
# the tutorial output is stable, but still save the full logs for traceability.
buf_out = io.StringIO()
buf_err = io.StringIO()
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    with contextlib.redirect_stdout(buf_out), contextlib.redirect_stderr(buf_err):
        fpp = btv.calculate_fpp(
            cache=cache,
            tic_id=TIC_ID,
            period=PERIOD_DAYS,
            t0=T0_BTJD,
            depth_ppm=float(candidate.depth_ppm),
            duration_hours=DURATION_HOURS,
            preset=FPP_PRESET,
            seed=FPP_SEED,
            replicates=FPP_REPLICATES,
            contrast_curve=contrast_curve,
        )

raw_log = buf_out.getvalue() + '\n\n--- STDERR ---\n\n' + buf_err.getvalue()

# Store the full raw log for traceability.
log_path = 'persistent_cache/fpp_triceratops_stdout.log'
with open(log_path, 'w', encoding='utf-8') as fp:
    fp.write(raw_log)

summary = {
    'disposition': fpp.get('disposition'),
    'fpp': fpp.get('fpp'),
    'nfpp': fpp.get('nfpp'),
    'prob_planet': fpp.get('prob_planet'),
    'prob_eb': fpp.get('prob_eb'),
    'prob_beb': fpp.get('prob_beb'),
    'sectors_used': fpp.get('sectors_used'),
    'n_nearby_sources': fpp.get('n_nearby_sources'),
    'runtime_seconds': fpp.get('runtime_seconds'),
    'triceratops_runtime': fpp.get('triceratops_runtime'),
    'run_seed': fpp.get('run_seed'),
    'base_seed': fpp.get('base_seed'),
    'replicates': fpp.get('replicates'),
    'log_path': log_path,
}

print(json.dumps(summary, indent=2, sort_keys=True))


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "base_seed": 123,
  "disposition": "VALIDATED",
  "fpp": 1e-06,
  "log_path": "persistent_cache/fpp_triceratops_stdout.log",
  "n_nearby_sources": 347,
  "nfpp": 0.0,
  "prob_beb": 0.0,
  "prob_eb": 0.0,
  "prob_planet": 0.959198,
  "replicates": 1,
  "run_seed": 123,
  "runtime_seconds": 111.6,
  "sectors_used": [
    55,
    75,
    82,
    83
  ],
  "triceratops_runtime": {
    "empirical_sigma_used": 0.00025361920376856806,
    "exptime_days": 0.001388881956700061,
    "external_filters": [],
    "flux_err_scalar_used": 0.00025361920376856806,
    "half_window_days": 0.33716666666666667,
    "max_points": 1500,
    "mc_draws": 50000,
    "min_flux_err": 5e-05,
    "n_external_lcs": 0,
    "n_points_raw": 73805,
    "n_points_used": 1500,
    "n_points_windowed": 3885,
    "use_empirical_noise_floor": true,
    "window_duration_mult": 2.0
  }
}
```

</details>


<details>
<summary><b>Analysis</b></summary>

<ul>
<li><b>What matters:</b> <code>fpp</code> is the overall false positive probability; <code>nfpp</code> is the “nearby false positive probability” (signal likely on a neighbor rather than the target).</li>
<li><b>Result:</b> this run returns <code>fpp = 1e-06</code>, <code>nfpp = 0.0</code>, and <code>prob_planet ≈ 0.959</code> with disposition <code>VALIDATED</code> (seeded run; log saved to <code>persistent_cache/fpp_triceratops_stdout.log</code>).</li>
<li><b>Interpretation:</b> given (a) the strong improvement under per-sector detrending (V12/V16/V19) and (b) AO constraints, these FPP/NFPP numbers strongly support statistical validation. Remaining due diligence is pixel/host robustness for this bright target.</li>
<li><b>Next:</b> generate a final consolidated validation packet (key metrics + reproducibility paths).</li>
</ul>

</details>


## 12) Consolidated validation packet (for audit/review)

This final cell collects the key deterministic metrics we computed in this notebook (robustness reruns + FPP/NFPP) into a single JSON object, and writes it to disk.


In [ ]:
packet = {
    'target': {
        'tic_id': TIC_ID,
        'toi': TOI_LABEL,
        'ra_deg': RA_DEG,
        'dec_deg': DEC_DEG,
    },
    'ephemeris': {
        'period_days': PERIOD_DAYS,
        't0_btjd': T0_BTJD,
        'duration_hours': DURATION_HOURS,
    },
    'depth_ppm': {
        'pdcsap_stitched_hat': float(candidate.depth_ppm),
        'detrended_stitched_hat': float(candidate_det.depth_ppm),
        'detrended_stitched_err': float(DEPTH_ERR_PPM_DET),
    },
    'robustness_detrended': {
        'v12_sweet': session_det.run('V12').metrics,
        'v16_model_competition': session_ext_det.run('V16').metrics,
        'v17_phase_shift_null': session_ext_det.run('V17').metrics,
        'v19_alias_diagnostics': session_ext_det.run('V19').metrics,
    },
    'fpp_ao': {
        'preset': FPP_PRESET,
        'seed': FPP_SEED,
        'replicates': FPP_REPLICATES,
        'contrast_curve_path': str(contrast_curve_path),
        'contrast_curve_filter': contrast_curve.filter,
        'summary': summary,
    },
}

packet_path = 'persistent_cache/toi5807_validation_packet.json'
with open(packet_path, 'w', encoding='utf-8') as fp:
    fp.write(json.dumps(packet, indent=2, sort_keys=True))

print(json.dumps({'packet_path': packet_path, 'packet_keys': sorted(packet.keys())}, indent=2, sort_keys=True))


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "packet_keys": [
    "depth_ppm",
    "ephemeris",
    "fpp_ao",
    "robustness_detrended",
    "target"
  ],
  "packet_path": "persistent_cache/toi5807_validation_packet.json"
}
```

</details>
